In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torchmetrics
!pip install statsmodels
!pip install datasets
!pip install SimpleITK
!pip install medpy
!pip install hausdorff

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.5/869.5 kB 22.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.3/156.3 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for medpy: filename=MedPy-0.5.2-cp310-cp310-linux_x86_64.whl size=762835 sha256=86118d5a5dc2ad5ec45d969d7982d76db966a6eba8051baa34f521dd27834a89
  Stored in directory: /root/.cache/pip/wheels/a1/b8/63/bdf557940ec60d1b8822e73ff9fbe

In [ ]:
import pandas as pd
import numpy as np
from decimal import getcontext, Decimal

np.set_printoptions(precision=25)
getcontext().prec = 25
pd.options.display.float_format = '{:.25f}'.format  # Adjust the format as needed
# Initialize a dictio# Set precision for numpy
np.set_printoptions(precision=25)

# Set precision for pandas
pd.set_option('display.float_format', '{:.25f}'.format)

# Torch 15 pic

In [ ]:
# Torch 15 pic

import pandas as pd
import numpy as np
import torch
import torchmetrics
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import nibabel as nib
import os
from decimal import getcontext, Decimal
from torchmetrics.functional import dice

np.set_printoptions(precision=25)
getcontext().prec = 25
pd.options.display.float_format = '{:.25f}'.format  # Adjust the format as needed

# Set precision for numpy
np.set_printoptions(precision=25)

# Set precision for pandas
pd.set_option('display.float_format', '{:.25f}'.format)

# SegmentationDataset Class
# SegmentationDataset Class
class SegmentationDataset(Dataset):
    def __init__(self, true_seg_dir, pred_seg_dir, transform=None):
        self.true_seg_dir = true_seg_dir
        self.pred_seg_dir = pred_seg_dir
        self.transform = transform

        # Get all files in directories
        self.true_seg_files = sorted(os.listdir(true_seg_dir))
        self.pred_seg_files = sorted(os.listdir(pred_seg_dir))

        # Extract the common parts (e.g. #LIDC-IDRI#LIDC-IDRI-0001#VISIT 1#.nii.gz)
        self.true_seg_base_names = [self.extract_base_name(file) for file in self.true_seg_files]
        self.pred_seg_base_names = [self.extract_base_name(file) for file in self.pred_seg_files]

        # Ensure that there are matching filenames between true and predicted
        assert set(self.true_seg_base_names) == set(self.pred_seg_base_names), \
            "True and predicted segmentation filenames do not match."

    def __len__(self):
        return len(self.true_seg_files)

    def __getitem__(self, idx):
        # Find the matching true and predicted segmentation files
        base_name = self.true_seg_base_names[idx]
        true_seg_filename = [file for file in self.true_seg_files if self.extract_base_name(file) == base_name][0]
        pred_seg_filename = [file for file in self.pred_seg_files if self.extract_base_name(file) == base_name][0]

        true_seg_path = os.path.join(self.true_seg_dir, true_seg_filename)
        pred_seg_path = os.path.join(self.pred_seg_dir, pred_seg_filename)

        true_seg_image = nib.load(true_seg_path).get_fdata()
        pred_seg_image = nib.load(pred_seg_path).get_fdata()

        # Ensure segmentation masks are binary (if needed)
        true_seg_image = np.expand_dims(np.clip(true_seg_image, 0, 1), axis=0)
        pred_seg_image = np.expand_dims(np.clip(pred_seg_image, 0, 1), axis=0)

        sample = {'true_seg': true_seg_image, 'pred_seg': pred_seg_image}

        if self.transform:
            sample = self.transform(sample)

        return sample

    def extract_base_name(self, file_name):
        base_name = file_name.split('#', 1)[1]
        return base_name


# ToTensor Class
class ToTensor(object):
    def __call__(self, sample):
        true_seg = torch.from_numpy(sample['true_seg'])
        pred_seg = torch.from_numpy(sample['pred_seg'])
        true_seg, pred_seg = true_seg.type(torch.float32), pred_seg.type(torch.float32)

        x, y, z = 256,256, 128  # Example target shape (D, H, W)
        true_seg_resized = F.interpolate(true_seg.unsqueeze(0), size=(x, y, z), mode='trilinear', align_corners=False).squeeze(0)
        pred_seg_resized = F.interpolate(pred_seg.unsqueeze(0), size=(x, y, z), mode='trilinear', align_corners=False).squeeze(0)

        return {'true_seg': true_seg_resized, 'pred_seg': pred_seg_resized}

# Example usage
true_segmentations_dir = '/content/drive/MyDrive/Loss Function Evaluation/Data/Segmentation/true'
predicted_masks_dir = '/content/drive/MyDrive/Loss Function Evaluation/Data/Segmentation/pred'
# Create dataset and dataloader
dataset = SegmentationDataset(true_seg_dir=true_segmentations_dir, pred_seg_dir=predicted_masks_dir, transform=ToTensor())
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)

# Initialize the results dictionary
results2 = {}

# Iterate through the dataloader and calculate metrics
for i, sample in enumerate(dataloader):
    y_true = sample['true_seg']
    y_pred = sample['pred_seg']

     # Ensure unique samples are being loaded
    print(f"Processing Sample {i}: y_true.shape = {y_true.shape}, y_pred.shape = {y_pred.shape}")

    # Clone and detach the tensors to avoid any computational graph tracking
    y_true = y_true.clone().detach().type(torch.float32)
    y_pred = y_pred.clone().detach().type(torch.float32)

    # Flatten the images for certain metrics
    y_true_flat = y_true.flatten()
    y_pred_flat = y_pred.flatten()

    # PyTorch metrics
    y_true_torch = y_true_flat.type(torch.int)
    y_pred_torch = y_pred_flat.type(torch.int)

    accuracy_metric = torchmetrics.Accuracy(task="binary")
    precision_metric = torchmetrics.Precision(task="binary")
    recall_metric = torchmetrics.Recall(task="binary")
    f1_metric = torchmetrics.F1Score(task="binary")
    iou_metric = torchmetrics.JaccardIndex(task="binary")

    accuracy_metric.update(y_pred_torch, y_true_torch)
    precision_metric.update(y_pred_torch, y_true_torch)
    recall_metric.update(y_pred_torch, y_true_torch)
    f1_metric.update(y_pred_torch, y_true_torch)
    iou_metric.update(y_pred_torch, y_true_torch)

    pt_accuracy = accuracy_metric.compute().item()
    pt_precision = precision_metric.compute().item()
    pt_recall = recall_metric.compute().item()
    pt_f1 = f1_metric.compute().item()
    pt_iou = iou_metric.compute().item()

    # Boundary F1 Score and Hausdorff Distance functions
    def extract_boundaries_torch(mask):
        """Extract boundary pixels from a binary segmentation mask using PyTorch."""
        # Ensure the mask has 5 dimensions: [batch_size, channels, depth, height, width] for 3D images
        if len(mask.shape) == 3:  # If the mask is 3D, expand it
            mask = mask.unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions

        # Apply max_pool2d for 2D or 3D pooling (in your case, max_pool3d)
        if mask.dim() == 5:  # For 3D images
            eroded_mask = F.max_pool3d(mask, kernel_size=3, stride=1, padding=1).squeeze()
        else:  # For 2D images (if applicable)
            eroded_mask = F.max_pool2d(mask, kernel_size=3, stride=1, padding=1).squeeze()

        # Boundary is the difference between the original mask and eroded mask
        boundary = (mask.squeeze() != eroded_mask).float()
        return boundary


    def custom_dilation_torch(mask, kernel_size=3):
        """Dilate the boundary pixels for 3D data."""
        kernel = torch.ones((1, 1, kernel_size, kernel_size, kernel_size), dtype=torch.float32)  # 3D kernel
        mask = mask.unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions if not present

        # Use conv3d instead of conv2d since we are dealing with 3D data
        dilated = F.conv3d(mask, kernel, padding=kernel_size // 2).squeeze()

        dilated = (dilated > 0).float()
        return dilated


    def boundary_f1_score_torch(y_true, y_pred, dilation_radius=1):
        y_true_boundary = extract_boundaries_torch(y_true)
        y_pred_boundary = extract_boundaries_torch(y_pred)
        y_true_boundary_dilated = custom_dilation_torch(y_true_boundary, kernel_size=2*dilation_radius+1)
        y_pred_boundary_dilated = custom_dilation_torch(y_pred_boundary, kernel_size=2*dilation_radius+1)
        y_true_boundary_flat = y_true_boundary_dilated.flatten()
        y_pred_boundary_flat = y_pred_boundary_dilated.flatten()
        precision_metric = torchmetrics.Precision(task="binary")
        recall_metric = torchmetrics.Recall(task="binary")
        precision_metric.update(y_pred_boundary_flat, y_true_boundary_flat)
        recall_metric.update(y_pred_boundary_flat, y_true_boundary_flat)
        precision_value = precision_metric.compute().item()
        recall_value = recall_metric.compute().item()
        f1_value = 2 * (precision_value * recall_value) / (precision_value + recall_value + 1e-6)
        return f1_value

    bf_score_torch = boundary_f1_score_torch(y_true, y_pred)

    def hausdorff_distance_torch(y_true, y_pred):
        y_true = torch.tensor(y_true, dtype=torch.float32)
        y_pred = torch.tensor(y_pred, dtype=torch.float32)
        y_true_points = torch.nonzero(y_true, as_tuple=False).float()
        y_pred_points = torch.nonzero(y_pred, as_tuple=False).float()
        if y_true_points.size(0) == 0 or y_pred_points.size(0) == 0:
            return float('inf')
        dists = torch.cdist(y_true_points.unsqueeze(0), y_pred_points.unsqueeze(0)).squeeze(0)
        forward_hausdorff = torch.max(torch.min(dists, dim=1)[0])
        backward_hausdorff = torch.max(torch.min(dists, dim=0)[0])
        hd = torch.max(forward_hausdorff, backward_hausdorff)
        return hd.item()

    hd_torch = hausdorff_distance_torch(y_true, y_pred)

    # Save all results
    results2[f'Sample {i}'] = {
        'Accuracy': pt_accuracy,
        'Precision': pt_precision,
        'Recall': pt_recall,
        'F1 Score': pt_f1,
        'IoU': pt_iou,
        'Dice Coefficient': dice(y_pred_torch, y_true_torch).item(),
        'BF Score': bf_score_torch,
        'Hausdorff Distance': hd_torch
    }

# Convert results to a DataFrame for better readability
results2_df = pd.DataFrame(results2).T
results2_df.index.name = 'Sample'
results2_df.reset_index(inplace=True)
print(results2_df)



# Save the DataFrame to an Excel file in the 'results' folder
results2_df.to_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Segmentation/binary_TORCH15_metrics.xlsx', index=False)
# Torch 15 pic

Processing Sample 0: y_true.shape = torch.Size([1, 1, 256, 256, 128]), y_pred.shape = torch.Size([1, 1, 256, 256, 128])


<ipython-input-6-14896c02852b>:183: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_true = torch.tensor(y_true, dtype=torch.float32)
<ipython-input-6-14896c02852b>:184: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_pred = torch.tensor(y_pred, dtype=torch.float32)


Processing Sample 1: y_true.shape = torch.Size([1, 1, 256, 256, 128]), y_pred.shape = torch.Size([1, 1, 256, 256, 128])
Processing Sample 2: y_true.shape = torch.Size([1, 1, 256, 256, 128]), y_pred.shape = torch.Size([1, 1, 256, 256, 128])
Processing Sample 3: y_true.shape = torch.Size([1, 1, 256, 256, 128]), y_pred.shape = torch.Size([1, 1, 256, 256, 128])
Processing Sample 4: y_true.shape = torch.Size([1, 1, 256, 256, 128]), y_pred.shape = torch.Size([1, 1, 256, 256, 128])
Processing Sample 5: y_true.shape = torch.Size([1, 1, 256, 256, 128]), y_pred.shape = torch.Size([1, 1, 256, 256, 128])
Processing Sample 6: y_true.shape = torch.Size([1, 1, 256, 256, 128]), y_pred.shape = torch.Size([1, 1, 256, 256, 128])
Processing Sample 7: y_true.shape = torch.Size([1, 1, 256, 256, 128]), y_pred.shape = torch.Size([1, 1, 256, 256, 128])
Processing Sample 8: y_true.shape = torch.Size([1, 1, 256, 256, 128]), y_pred.shape = torch.Size([1, 1, 256, 256, 128])
Processing Sample 9: y_true.shape = torc

# SimpleITK 0-29

In [ ]:
# SimpleITK 0-29

import os
import numpy as np
import pandas as pd
import nibabel as nib
import torch
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, jaccard_score, cohen_kappa_score
import SimpleITK as sitk
from medpy.metric.binary import hd, dc, precision as med_precision, recall as med_recall

np.set_printoptions(precision=25)
pd.options.display.float_format = '{:.25f}'.format

# SegmentationDataset Class (PyTorch)
class SegmentationDataset(Dataset):
    def __init__(self, true_seg_dir, pred_seg_dir, transform=None):
        self.true_seg_dir = true_seg_dir
        self.pred_seg_dir = pred_seg_dir
        self.transform = transform

        # Get all files in directories
        self.true_seg_files = sorted(os.listdir(true_seg_dir))
        self.pred_seg_files = sorted(os.listdir(pred_seg_dir))

        # Extract the common parts (e.g. #LIDC-IDRI#LIDC-IDRI-0001#VISIT 1#.nii.gz)
        self.true_seg_base_names = [self.extract_base_name(file) for file in self.true_seg_files]
        self.pred_seg_base_names = [self.extract_base_name(file) for file in self.pred_seg_files]

        # Ensure that there are matching filenames between true and predicted
        assert set(self.true_seg_base_names) == set(self.pred_seg_base_names), \
            "True and predicted segmentation filenames do not match."

    def __len__(self):
        return len(self.true_seg_files)

    def __getitem__(self, idx):
        base_name = self.true_seg_base_names[idx]
        true_seg_filename = [file for file in self.true_seg_files if self.extract_base_name(file) == base_name][0]
        pred_seg_filename = [file for file in self.pred_seg_files if self.extract_base_name(file) == base_name][0]

        true_seg_path = os.path.join(self.true_seg_dir, true_seg_filename)
        pred_seg_path = os.path.join(self.pred_seg_dir, pred_seg_filename)

        true_seg_image = nib.load(true_seg_path).get_fdata()
        pred_seg_image = nib.load(pred_seg_path).get_fdata()

        # Ensure segmentation masks are binary (0 or 1)
        true_seg_image = np.expand_dims(np.clip(true_seg_image, 0, 1), axis=0)
        pred_seg_image = np.expand_dims(np.clip(pred_seg_image, 0, 1), axis=0)

        sample = {'true_seg': true_seg_image, 'pred_seg': pred_seg_image}

        if self.transform:
            sample = self.transform(sample)

        return sample

    def extract_base_name(self, file_name):
        base_name = file_name.split('#', 1)[1]
        return base_name

class ToTensorAndResize(object):
    def __call__(self, sample):
        true_seg = torch.from_numpy(sample['true_seg']).float()
        pred_seg = torch.from_numpy(sample['pred_seg']).float()

        # Reshape the tensors to target size of 128x128x128 (D, H, W)
        true_seg_resized = self.resize_3d(true_seg, (128, 128, 128))
        pred_seg_resized = self.resize_3d(pred_seg, (128, 128, 128))

        return {'true_seg': true_seg_resized, 'pred_seg': pred_seg_resized}

    def resize_3d(self, volume, target_shape):
        """
        Resize a 3D PyTorch tensor to a target shape using trilinear interpolation.
        :param volume: Input 3D PyTorch tensor
        :param target_shape: Desired shape (D, H, W)
        :return: Resized 3D PyTorch tensor
        """
        # Ensure the tensor has at least 3 spatial dimensions
        if len(volume.shape) == 3:  # Shape [D, H, W]
            volume = volume.unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions [1, 1, D, H, W]
        elif len(volume.shape) == 4:  # Shape [C, D, H, W]
            volume = volume.unsqueeze(0)  # Add batch dimension

        # Check the shape before interpolation to make sure it's valid
        if volume.dim() != 5:
            raise ValueError(f"Expected a 5D tensor, but got {volume.dim()}D tensor with shape {volume.shape}")

        # Perform the interpolation (resizing)
        resized_volume = torch.nn.functional.interpolate(volume, size=target_shape, mode='trilinear', align_corners=False)

        # Remove the batch and channel dimensions
        return resized_volume.squeeze(0).squeeze(0)  # Back to shape [D, H, W]

# Load data in PyTorch
true_segmentations_dir = '/content/drive/MyDrive/Mr-Alizadeh/Train-760/CT-segmentation'
predicted_masks_dir = '/content/drive/MyDrive/Mr-Alizadeh/Reconnet_predicted_masks'

# Create dataset (PyTorch)
dataset = SegmentationDataset(true_seg_dir=true_segmentations_dir, pred_seg_dir=predicted_masks_dir, transform=ToTensorAndResize())

# Initialize the results dictionary
results = {}

# SimpleITK
for i in range(30):
    sample = dataset[i]

    # Convert PyTorch tensors to NumPy arrays
    y_true_np = sample['true_seg'].numpy().astype(np.int32).flatten()
    y_pred_np = sample['pred_seg'].numpy().astype(np.int32).flatten()
        # بررسی اینکه تصاویر واقعی یا پیش‌بینی‌شده خالی نیستند
    if np.count_nonzero(y_true_np) == 0:
        print(f"True segmentation is empty for Sample {i}, setting metrics to NaN.")
        sklearn_accuracy = np.nan
        sklearn_precision = np.nan
        sklearn_recall = np.nan
        # سایر متریک‌ها را با NaN پر کنید...
        continue

    if np.count_nonzero(y_pred_np) == 0:
        print(f"Predicted segmentation is empty for Sample {i}, setting metrics to NaN.")
        sklearn_accuracy = np.nan
        sklearn_precision = np.nan
        sklearn_recall = np.nan
        # سایر متریک‌ها را با NaN پر کنید...
        continue
    # بررسی اینکه تصاویر خالی نیستند
    if np.count_nonzero(y_true_np) == 0:
        raise ValueError(f"True segmentation is empty for Sample {i}")

    if np.count_nonzero(y_pred_np) == 0:
        raise ValueError(f"Predicted segmentation is empty for Sample {i}")

    # بررسی تطابق ابعاد تصاویر
    if sample['true_seg'].shape != sample['pred_seg'].shape:
        raise ValueError(f"Shape mismatch: True segmentation shape {sample['true_seg'].shape}, "
                         f"Predicted segmentation shape {sample['pred_seg'].shape}")

    # Metrics using SimpleITK (Cast to integer type)
    true_itk = sitk.Cast(sitk.GetImageFromArray(sample['true_seg'].numpy().astype(np.uint8).squeeze()), sitk.sitkUInt8)
    pred_itk = sitk.Cast(sitk.GetImageFromArray(sample['pred_seg'].numpy().astype(np.uint8).squeeze()), sitk.sitkUInt8)

    dice_itk = sitk.LabelOverlapMeasuresImageFilter()
    dice_itk.Execute(true_itk, pred_itk)
    dice_itk = dice_itk.GetDiceCoefficient()
    # محاسبه Hausdorff Distance در صورت غیر خالی بودن تصاویر
    if np.count_nonzero(y_true_np) > 0 and np.count_nonzero(y_pred_np) > 0:
        hd_itk = sitk.HausdorffDistanceImageFilter()
        hd_itk.Execute(true_itk, pred_itk)
        hd_itk_value = hd_itk.GetHausdorffDistance()
    else:
        hd_itk_value = np.nan  # یا مقدار پیش‌فرض
    # hd_itk = sitk.HausdorffDistanceImageFilter()
    # hd_itk.Execute(true_itk, pred_itk)
    # hd_itk = hd_itk.GetHausdorffDistance()

    # Store the metrics
    results[f'SITK_Sample {i}'] = {
            'Dice Coefficient': dice_itk,
            'Hausdorff Distance': hd_itk
    }
# Convert results to a DataFrame for better readability
results_df = pd.DataFrame(results).T
results_df.index.name = 'Sample'
results_df.reset_index(inplace=True)
print(results_df)

# Save the DataFrame to an Excel file
results_df.to_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Segmentation/binary_SITK_0-29.xlsx', index=False)
# SimpleITK 0-29

Predicted segmentation is empty for Sample 2, setting metrics to NaN.
Predicted segmentation is empty for Sample 3, setting metrics to NaN.
True segmentation is empty for Sample 5, setting metrics to NaN.
Predicted segmentation is empty for Sample 13, setting metrics to NaN.
True segmentation is empty for Sample 19, setting metrics to NaN.
Predicted segmentation is empty for Sample 25, setting metrics to NaN.
            Sample            Dice Coefficient  \
0    SITK_Sample 0 0.6367924528301887043824081   
1    SITK_Sample 1 0.6359447004608295150873687   
2    SITK_Sample 4 0.6180422264875240490056285   
3    SITK_Sample 6 0.8000000000000000444089210   
4    SITK_Sample 7 0.5789473684210525439297612   
5    SITK_Sample 8 0.0129870129870129878690843   
6    SITK_Sample 9 0.7022900763358779219203143   
7   SITK_Sample 10 0.6077348066298342565971780   
8   SITK_Sample 11 0.1538461538461538546940233   
9   SITK_Sample 12 0.7708333333333332593184650   
10  SITK_Sample 14 0.5473684210526316

# SimpleITK 30-59

In [ ]:
# SimpleITK 30-59

import os
import numpy as np
import pandas as pd
import nibabel as nib
import torch
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, jaccard_score, cohen_kappa_score
import SimpleITK as sitk
from medpy.metric.binary import hd, dc, precision as med_precision, recall as med_recall

np.set_printoptions(precision=25)
pd.options.display.float_format = '{:.25f}'.format

# SegmentationDataset Class (PyTorch)
class SegmentationDataset(Dataset):
    def __init__(self, true_seg_dir, pred_seg_dir, transform=None):
        self.true_seg_dir = true_seg_dir
        self.pred_seg_dir = pred_seg_dir
        self.transform = transform

        # Get all files in directories
        self.true_seg_files = sorted(os.listdir(true_seg_dir))
        self.pred_seg_files = sorted(os.listdir(pred_seg_dir))

        # Extract the common parts (e.g. #LIDC-IDRI#LIDC-IDRI-0001#VISIT 1#.nii.gz)
        self.true_seg_base_names = [self.extract_base_name(file) for file in self.true_seg_files]
        self.pred_seg_base_names = [self.extract_base_name(file) for file in self.pred_seg_files]

        # Ensure that there are matching filenames between true and predicted
        assert set(self.true_seg_base_names) == set(self.pred_seg_base_names), \
            "True and predicted segmentation filenames do not match."

    def __len__(self):
        return len(self.true_seg_files)

    def __getitem__(self, idx):
        base_name = self.true_seg_base_names[idx]
        true_seg_filename = [file for file in self.true_seg_files if self.extract_base_name(file) == base_name][0]
        pred_seg_filename = [file for file in self.pred_seg_files if self.extract_base_name(file) == base_name][0]

        true_seg_path = os.path.join(self.true_seg_dir, true_seg_filename)
        pred_seg_path = os.path.join(self.pred_seg_dir, pred_seg_filename)

        true_seg_image = nib.load(true_seg_path).get_fdata()
        pred_seg_image = nib.load(pred_seg_path).get_fdata()

        # Ensure segmentation masks are binary (0 or 1)
        true_seg_image = np.expand_dims(np.clip(true_seg_image, 0, 1), axis=0)
        pred_seg_image = np.expand_dims(np.clip(pred_seg_image, 0, 1), axis=0)

        sample = {'true_seg': true_seg_image, 'pred_seg': pred_seg_image}

        if self.transform:
            sample = self.transform(sample)

        return sample

    def extract_base_name(self, file_name):
        base_name = file_name.split('#', 1)[1]
        return base_name

class ToTensorAndResize(object):
    def __call__(self, sample):
        true_seg = torch.from_numpy(sample['true_seg']).float()
        pred_seg = torch.from_numpy(sample['pred_seg']).float()

        # Reshape the tensors to target size of 128x128x128 (D, H, W)
        true_seg_resized = self.resize_3d(true_seg, (128, 128, 128))
        pred_seg_resized = self.resize_3d(pred_seg, (128, 128, 128))

        return {'true_seg': true_seg_resized, 'pred_seg': pred_seg_resized}

    def resize_3d(self, volume, target_shape):
        """
        Resize a 3D PyTorch tensor to a target shape using trilinear interpolation.
        :param volume: Input 3D PyTorch tensor
        :param target_shape: Desired shape (D, H, W)
        :return: Resized 3D PyTorch tensor
        """
        # Ensure the tensor has at least 3 spatial dimensions
        if len(volume.shape) == 3:  # Shape [D, H, W]
            volume = volume.unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions [1, 1, D, H, W]
        elif len(volume.shape) == 4:  # Shape [C, D, H, W]
            volume = volume.unsqueeze(0)  # Add batch dimension

        # Check the shape before interpolation to make sure it's valid
        if volume.dim() != 5:
            raise ValueError(f"Expected a 5D tensor, but got {volume.dim()}D tensor with shape {volume.shape}")

        # Perform the interpolation (resizing)
        resized_volume = torch.nn.functional.interpolate(volume, size=target_shape, mode='trilinear', align_corners=False)

        # Remove the batch and channel dimensions
        return resized_volume.squeeze(0).squeeze(0)  # Back to shape [D, H, W]

# Load data in PyTorch
true_segmentations_dir = '/content/drive/MyDrive/Mr-Alizadeh/Train-760/CT-segmentation'
predicted_masks_dir = '/content/drive/MyDrive/Mr-Alizadeh/Reconnet_predicted_masks'

# Create dataset (PyTorch)
dataset = SegmentationDataset(true_seg_dir=true_segmentations_dir, pred_seg_dir=predicted_masks_dir, transform=ToTensorAndResize())

# Initialize the results dictionary
results = {}

# SimpleITK
for i in range(30,60):
    sample = dataset[i]

    # Convert PyTorch tensors to NumPy arrays
    y_true_np = sample['true_seg'].numpy().astype(np.int32).flatten()
    y_pred_np = sample['pred_seg'].numpy().astype(np.int32).flatten()
        # بررسی اینکه تصاویر واقعی یا پیش‌بینی‌شده خالی نیستند
    if np.count_nonzero(y_true_np) == 0:
        print(f"True segmentation is empty for Sample {i}, setting metrics to NaN.")
        sklearn_accuracy = np.nan
        sklearn_precision = np.nan
        sklearn_recall = np.nan
        # سایر متریک‌ها را با NaN پر کنید...
        continue

    if np.count_nonzero(y_pred_np) == 0:
        print(f"Predicted segmentation is empty for Sample {i}, setting metrics to NaN.")
        sklearn_accuracy = np.nan
        sklearn_precision = np.nan
        sklearn_recall = np.nan
        # سایر متریک‌ها را با NaN پر کنید...
        continue
    # بررسی اینکه تصاویر خالی نیستند
    if np.count_nonzero(y_true_np) == 0:
        raise ValueError(f"True segmentation is empty for Sample {i}")

    if np.count_nonzero(y_pred_np) == 0:
        raise ValueError(f"Predicted segmentation is empty for Sample {i}")

    # بررسی تطابق ابعاد تصاویر
    if sample['true_seg'].shape != sample['pred_seg'].shape:
        raise ValueError(f"Shape mismatch: True segmentation shape {sample['true_seg'].shape}, "
                         f"Predicted segmentation shape {sample['pred_seg'].shape}")

    # Metrics using SimpleITK (Cast to integer type)
    true_itk = sitk.Cast(sitk.GetImageFromArray(sample['true_seg'].numpy().astype(np.uint8).squeeze()), sitk.sitkUInt8)
    pred_itk = sitk.Cast(sitk.GetImageFromArray(sample['pred_seg'].numpy().astype(np.uint8).squeeze()), sitk.sitkUInt8)

    dice_itk = sitk.LabelOverlapMeasuresImageFilter()
    dice_itk.Execute(true_itk, pred_itk)
    dice_itk = dice_itk.GetDiceCoefficient()
    # محاسبه Hausdorff Distance در صورت غیر خالی بودن تصاویر
    if np.count_nonzero(y_true_np) > 0 and np.count_nonzero(y_pred_np) > 0:
        hd_itk = sitk.HausdorffDistanceImageFilter()
        hd_itk.Execute(true_itk, pred_itk)
        hd_itk_value = hd_itk.GetHausdorffDistance()
    else:
        hd_itk_value = np.nan  # یا مقدار پیش‌فرض
    # hd_itk = sitk.HausdorffDistanceImageFilter()
    # hd_itk.Execute(true_itk, pred_itk)
    # hd_itk = hd_itk.GetHausdorffDistance()

    # Store the metrics
    results[f'SITK_Sample {i}'] = {
            'Dice Coefficient': dice_itk,
            'Hausdorff Distance': hd_itk
    }
# Convert results to a DataFrame for better readability
results_df = pd.DataFrame(results).T
results_df.index.name = 'Sample'
results_df.reset_index(inplace=True)
print(results_df)

# Save the DataFrame to an Excel file
results_df.to_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Segmentation/binary_SITK_30-59.xlsx', index=False)
# SimpleITK 30-59

Predicted segmentation is empty for Sample 38, setting metrics to NaN.
True segmentation is empty for Sample 50, setting metrics to NaN.
            Sample            Dice Coefficient  \
0   SITK_Sample 30 0.6093750000000000000000000   
1   SITK_Sample 31 0.5454545454545454141737082   
2   SITK_Sample 32 0.8487804878048780254928829   
3   SITK_Sample 33 0.5937500000000000000000000   
4   SITK_Sample 34 0.6153846153846153077537906   
5   SITK_Sample 35 0.7843866171003718301690810   
6   SITK_Sample 36 0.7441860465116278966490881   
7   SITK_Sample 37 0.3800000000000000044408921   
8   SITK_Sample 39 0.7588932806324111268736488   
9   SITK_Sample 40 0.6666666666666666296592325   
10  SITK_Sample 41 0.9023090586145647851168405   
11  SITK_Sample 42 0.5561694290976059162900924   
12  SITK_Sample 43 0.6939611054247698529451327   
13  SITK_Sample 44 0.5365853658536584580573958   
14  SITK_Sample 45 0.7742857142857143548653198   
15  SITK_Sample 46 0.0000000000000000000000000   
16  SITK_Samp

# SimpleITK 60-99

In [ ]:
# SimpleITK 60-99

import os
import numpy as np
import pandas as pd
import nibabel as nib
import torch
from torch.utils.data import Dataset
import SimpleITK as sitk

np.set_printoptions(precision=25)
pd.options.display.float_format = '{:.25f}'.format

# SegmentationDataset Class (PyTorch)
class SegmentationDataset(Dataset):
    def __init__(self, true_seg_dir, pred_seg_dir, transform=None):
        self.true_seg_dir = true_seg_dir
        self.pred_seg_dir = pred_seg_dir
        self.transform = transform

        # Get all files in directories
        self.true_seg_files = sorted(os.listdir(true_seg_dir))
        self.pred_seg_files = sorted(os.listdir(pred_seg_dir))

        # Extract the common parts (e.g. #LIDC-IDRI#LIDC-IDRI-0001#VISIT 1#.nii.gz)
        self.true_seg_base_names = [self.extract_base_name(file) for file in self.true_seg_files]
        self.pred_seg_base_names = [self.extract_base_name(file) for file in self.pred_seg_files]

        # Ensure that there are matching filenames between true and predicted
        assert set(self.true_seg_base_names) == set(self.pred_seg_base_names), \
            "True and predicted segmentation filenames do not match."

    def __len__(self):
        return len(self.true_seg_files)

    def __getitem__(self, idx):
        base_name = self.true_seg_base_names[idx]
        true_seg_filename = [file for file in self.true_seg_files if self.extract_base_name(file) == base_name][0]
        pred_seg_filename = [file for file in self.pred_seg_files if self.extract_base_name(file) == base_name][0]

        true_seg_path = os.path.join(self.true_seg_dir, true_seg_filename)
        pred_seg_path = os.path.join(self.pred_seg_dir, pred_seg_filename)

        true_seg_image = nib.load(true_seg_path).get_fdata()
        pred_seg_image = nib.load(pred_seg_path).get_fdata()

        # Ensure segmentation masks are binary (0 or 1)
        true_seg_image = np.expand_dims(np.clip(true_seg_image, 0, 1), axis=0)
        pred_seg_image = np.expand_dims(np.clip(pred_seg_image, 0, 1), axis=0)

        sample = {'true_seg': true_seg_image, 'pred_seg': pred_seg_image}

        if self.transform:
            sample = self.transform(sample)

        return sample

    def extract_base_name(self, file_name):
        base_name = file_name.split('#', 1)[1]
        return base_name

class ToTensorAndResize(object):
    def __call__(self, sample):
        true_seg = torch.from_numpy(sample['true_seg']).float()
        pred_seg = torch.from_numpy(sample['pred_seg']).float()

        # Reshape the tensors to target size of 128x128x128 (D, H, W)
        true_seg_resized = self.resize_3d(true_seg, (128, 128, 128))
        pred_seg_resized = self.resize_3d(pred_seg, (128, 128, 128))

        return {'true_seg': true_seg_resized, 'pred_seg': pred_seg_resized}

    def resize_3d(self, volume, target_shape):
        """
        Resize a 3D PyTorch tensor to a target shape using trilinear interpolation.
        :param volume: Input 3D PyTorch tensor
        :param target_shape: Desired shape (D, H, W)
        :return: Resized 3D PyTorch tensor
        """
        # Ensure the tensor has at least 3 spatial dimensions
        if len(volume.shape) == 3:  # Shape [D, H, W]
            volume = volume.unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions [1, 1, D, H, W]
        elif len(volume.shape) == 4:  # Shape [C, D, H, W]
            volume = volume.unsqueeze(0)  # Add batch dimension

        # Check the shape before interpolation to make sure it's valid
        if volume.dim() != 5:
            raise ValueError(f"Expected a 5D tensor, but got {volume.dim()}D tensor with shape {volume.shape}")

        # Perform the interpolation (resizing)
        resized_volume = torch.nn.functional.interpolate(volume, size=target_shape, mode='trilinear', align_corners=False)

        # Remove the batch and channel dimensions
        return resized_volume.squeeze(0).squeeze(0)  # Back to shape [D, H, W]

# Load data in PyTorch
true_segmentations_dir = '/content/drive/MyDrive/Mr-Alizadeh/Train-760/CT-segmentation'
predicted_masks_dir = '/content/drive/MyDrive/Mr-Alizadeh/Reconnet_predicted_masks'

# Create dataset (PyTorch)
dataset = SegmentationDataset(true_seg_dir=true_segmentations_dir, pred_seg_dir=predicted_masks_dir, transform=ToTensorAndResize())

# Initialize the results dictionary
results = {}

# SimpleITK
for i in range(60,100):
    sample = dataset[i]

    # Convert PyTorch tensors to NumPy arrays
    y_true_np = sample['true_seg'].numpy().astype(np.int32).flatten()
    y_pred_np = sample['pred_seg'].numpy().astype(np.int32).flatten()
        # بررسی اینکه تصاویر واقعی یا پیش‌بینی‌شده خالی نیستند
    if np.count_nonzero(y_true_np) == 0:
        print(f"True segmentation is empty for Sample {i}, setting metrics to NaN.")
        sklearn_accuracy = np.nan
        sklearn_precision = np.nan
        sklearn_recall = np.nan
        # سایر متریک‌ها را با NaN پر کنید...
        continue

    if np.count_nonzero(y_pred_np) == 0:
        print(f"Predicted segmentation is empty for Sample {i}, setting metrics to NaN.")
        sklearn_accuracy = np.nan
        sklearn_precision = np.nan
        sklearn_recall = np.nan
        # سایر متریک‌ها را با NaN پر کنید...
        continue
    # بررسی اینکه تصاویر خالی نیستند
    if np.count_nonzero(y_true_np) == 0:
        raise ValueError(f"True segmentation is empty for Sample {i}")

    if np.count_nonzero(y_pred_np) == 0:
        raise ValueError(f"Predicted segmentation is empty for Sample {i}")

    # بررسی تطابق ابعاد تصاویر
    if sample['true_seg'].shape != sample['pred_seg'].shape:
        raise ValueError(f"Shape mismatch: True segmentation shape {sample['true_seg'].shape}, "
                         f"Predicted segmentation shape {sample['pred_seg'].shape}")

    # Metrics using SimpleITK (Cast to integer type)
    true_itk = sitk.Cast(sitk.GetImageFromArray(sample['true_seg'].numpy().astype(np.uint8).squeeze()), sitk.sitkUInt8)
    pred_itk = sitk.Cast(sitk.GetImageFromArray(sample['pred_seg'].numpy().astype(np.uint8).squeeze()), sitk.sitkUInt8)

    dice_itk = sitk.LabelOverlapMeasuresImageFilter()
    dice_itk.Execute(true_itk, pred_itk)
    dice_itk = dice_itk.GetDiceCoefficient()
    # محاسبه Hausdorff Distance در صورت غیر خالی بودن تصاویر
    if np.count_nonzero(y_true_np) > 0 and np.count_nonzero(y_pred_np) > 0:
        hd_itk = sitk.HausdorffDistanceImageFilter()
        hd_itk.Execute(true_itk, pred_itk)
        hd_itk_value = hd_itk.GetHausdorffDistance()
    else:
        hd_itk_value = np.nan  # یا مقدار پیش‌فرض
    # hd_itk = sitk.HausdorffDistanceImageFilter()
    # hd_itk.Execute(true_itk, pred_itk)
    # hd_itk = hd_itk.GetHausdorffDistance()

    # Store the metrics
    results[f'SITK_Sample {i}'] = {
            'Dice Coefficient': dice_itk,
            'Hausdorff Distance': hd_itk
    }
# Convert results to a DataFrame for better readability
results_df = pd.DataFrame(results).T
results_df.index.name = 'Sample'
results_df.reset_index(inplace=True)
print(results_df)

# Save the DataFrame to an Excel file
results_df.to_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Segmentation/binary_SITK_60-99.xlsx', index=False)
# SimpleITK 60-99

Predicted segmentation is empty for Sample 63, setting metrics to NaN.
Predicted segmentation is empty for Sample 69, setting metrics to NaN.
True segmentation is empty for Sample 72, setting metrics to NaN.
Predicted segmentation is empty for Sample 74, setting metrics to NaN.
Predicted segmentation is empty for Sample 75, setting metrics to NaN.
True segmentation is empty for Sample 77, setting metrics to NaN.
Predicted segmentation is empty for Sample 85, setting metrics to NaN.
Predicted segmentation is empty for Sample 96, setting metrics to NaN.
Predicted segmentation is empty for Sample 97, setting metrics to NaN.
            Sample            Dice Coefficient  \
0   SITK_Sample 60 0.6611570247933884481028599   
1   SITK_Sample 61 0.8529411764705882026404993   
2   SITK_Sample 62 0.1999999999999999833466546   
3   SITK_Sample 64 0.6598360655737705027235052   
4   SITK_Sample 65 0.5809128630705393536715064   
5   SITK_Sample 66 0.8690807799442896719455121   
6   SITK_Sample 67 0.

# Medpy 0-29

In [ ]:
# Medpy 0-29

import os
import numpy as np
import pandas as pd
import nibabel as nib
import torch
from torch.utils.data import Dataset
from medpy.metric.binary import hd, dc, precision as med_precision, recall as med_recall

np.set_printoptions(precision=25)
pd.options.display.float_format = '{:.25f}'.format

# SegmentationDataset Class (PyTorch)
class SegmentationDataset(Dataset):
    def __init__(self, true_seg_dir, pred_seg_dir, transform=None):
        self.true_seg_dir = true_seg_dir
        self.pred_seg_dir = pred_seg_dir
        self.transform = transform

        # Get all files in directories
        self.true_seg_files = sorted(os.listdir(true_seg_dir))
        self.pred_seg_files = sorted(os.listdir(pred_seg_dir))

        # Extract the common parts (e.g. #LIDC-IDRI#LIDC-IDRI-0001#VISIT 1#.nii.gz)
        self.true_seg_base_names = [self.extract_base_name(file) for file in self.true_seg_files]
        self.pred_seg_base_names = [self.extract_base_name(file) for file in self.pred_seg_files]

        # Ensure that there are matching filenames between true and predicted
        assert set(self.true_seg_base_names) == set(self.pred_seg_base_names), \
            "True and predicted segmentation filenames do not match."

    def __len__(self):
        return len(self.true_seg_files)

    def __getitem__(self, idx):
        base_name = self.true_seg_base_names[idx]
        true_seg_filename = [file for file in self.true_seg_files if self.extract_base_name(file) == base_name][0]
        pred_seg_filename = [file for file in self.pred_seg_files if self.extract_base_name(file) == base_name][0]

        true_seg_path = os.path.join(self.true_seg_dir, true_seg_filename)
        pred_seg_path = os.path.join(self.pred_seg_dir, pred_seg_filename)

        true_seg_image = nib.load(true_seg_path).get_fdata()
        pred_seg_image = nib.load(pred_seg_path).get_fdata()

        # Ensure segmentation masks are binary (0 or 1)
        true_seg_image = np.expand_dims(np.clip(true_seg_image, 0, 1), axis=0)
        pred_seg_image = np.expand_dims(np.clip(pred_seg_image, 0, 1), axis=0)

        sample = {'true_seg': true_seg_image, 'pred_seg': pred_seg_image}

        if self.transform:
            sample = self.transform(sample)

        return sample

    def extract_base_name(self, file_name):
        base_name = file_name.split('#', 1)[1]
        return base_name

class ToTensorAndResize(object):
    def __call__(self, sample):
        true_seg = torch.from_numpy(sample['true_seg']).float()
        pred_seg = torch.from_numpy(sample['pred_seg']).float()

        # Reshape the tensors to target size of 128x128x128 (D, H, W)
        true_seg_resized = self.resize_3d(true_seg, (128, 128, 128))
        pred_seg_resized = self.resize_3d(pred_seg, (128, 128, 128))

        return {'true_seg': true_seg_resized, 'pred_seg': pred_seg_resized}

    def resize_3d(self, volume, target_shape):
        """
        Resize a 3D PyTorch tensor to a target shape using trilinear interpolation.
        :param volume: Input 3D PyTorch tensor
        :param target_shape: Desired shape (D, H, W)
        :return: Resized 3D PyTorch tensor
        """
        # Ensure the tensor has at least 3 spatial dimensions
        if len(volume.shape) == 3:  # Shape [D, H, W]
            volume = volume.unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions [1, 1, D, H, W]
        elif len(volume.shape) == 4:  # Shape [C, D, H, W]
            volume = volume.unsqueeze(0)  # Add batch dimension

        # Check the shape before interpolation to make sure it's valid
        if volume.dim() != 5:
            raise ValueError(f"Expected a 5D tensor, but got {volume.dim()}D tensor with shape {volume.shape}")

        # Perform the interpolation (resizing)
        resized_volume = torch.nn.functional.interpolate(volume, size=target_shape, mode='trilinear', align_corners=False)

        # Remove the batch and channel dimensions
        return resized_volume.squeeze(0).squeeze(0)  # Back to shape [D, H, W]

# Load data in PyTorch
true_segmentations_dir = '/content/drive/MyDrive/Mr-Alizadeh/Train-760/CT-segmentation'
predicted_masks_dir = '/content/drive/MyDrive/Mr-Alizadeh/Reconnet_predicted_masks'
# Create dataset (PyTorch)
dataset = SegmentationDataset(true_seg_dir=true_segmentations_dir, pred_seg_dir=predicted_masks_dir, transform=ToTensorAndResize())

# Initialize the results dictionary
results = {}

# Iterate through the dataset and calculate metrics using Scikit-learn, SimpleITK, and MedPy
for i in range(30):
    sample = dataset[i]

    # Convert PyTorch tensors to NumPy arrays
    y_true_np = sample['true_seg'].numpy().astype(np.int32).flatten()
    y_pred_np = sample['pred_seg'].numpy().astype(np.int32).flatten()
        # بررسی اینکه تصاویر واقعی یا پیش‌بینی‌شده خالی نیستند
    if np.count_nonzero(y_true_np) == 0:
        print(f"True segmentation is empty for Sample {i}, setting metrics to NaN.")
        sklearn_accuracy = np.nan
        sklearn_precision = np.nan
        sklearn_recall = np.nan
        # سایر متریک‌ها را با NaN پر کنید...
        continue

    if np.count_nonzero(y_pred_np) == 0:
        print(f"Predicted segmentation is empty for Sample {i}, setting metrics to NaN.")
        sklearn_accuracy = np.nan
        sklearn_precision = np.nan
        sklearn_recall = np.nan
        # سایر متریک‌ها را با NaN پر کنید...
        continue
    # بررسی اینکه تصاویر خالی نیستند
    if np.count_nonzero(y_true_np) == 0:
        raise ValueError(f"True segmentation is empty for Sample {i}")

    if np.count_nonzero(y_pred_np) == 0:
        raise ValueError(f"Predicted segmentation is empty for Sample {i}")

    # بررسی تطابق ابعاد تصاویر
    if sample['true_seg'].shape != sample['pred_seg'].shape:
        raise ValueError(f"Shape mismatch: True segmentation shape {sample['true_seg'].shape}, "
                         f"Predicted segmentation shape {sample['pred_seg'].shape}")


    # Metrics using MedPy
    dc_medpy = dc(sample['pred_seg'].numpy().astype(np.int32).flatten(), sample['true_seg'].numpy().astype(np.int32).flatten())
    hausdorff_distance_med = hd(sample['pred_seg'].numpy().astype(np.int32).flatten(), sample['true_seg'].numpy().astype(np.int32).flatten())
    precision_medpy = med_precision(sample['pred_seg'].numpy().astype(np.int32).flatten(), sample['true_seg'].numpy().astype(np.int32).flatten())
    recall_medpy = med_recall(sample['pred_seg'].numpy().astype(np.int32).flatten(), sample['true_seg'].numpy().astype(np.int32).flatten())

    # Manually calculate IoU for MedPy (Jaccard Index)
    tp = np.sum((y_true_np == 1) & (y_pred_np == 1))
    fp = np.sum((y_true_np == 0) & (y_pred_np == 1))
    fn = np.sum((y_true_np == 1) & (y_pred_np == 0))
    iou_medpy = tp / (tp + fp + fn + 1e-6)  # Add a small epsilon to avoid division by zero

    # Calculate F1 Score for MedPy
    f1_medpy = 2 * (precision_medpy * recall_medpy) / (precision_medpy + recall_medpy + 1e-6)

    # Store the metrics
    results[f'Med_Sample {i}'] = {
            'Dice Coefficient': dc_medpy,
            'Hausdorff Distance': hausdorff_distance_med,
            'IoU': iou_medpy,
            'Precision': precision_medpy,
            'Recall': recall_medpy,
            'F1 Score': f1_medpy
    }

# Convert results to a DataFrame for better readability
results_df = pd.DataFrame(results).T
results_df.index.name = 'Sample'
results_df.reset_index(inplace=True)
print(results_df)

# Save the DataFrame to an Excel file
results_df.to_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Segmentation/binary_Med_0-29.xlsx', index=False)
# Medpy 0-29

Predicted segmentation is empty for Sample 2, setting metrics to NaN.
Predicted segmentation is empty for Sample 3, setting metrics to NaN.
True segmentation is empty for Sample 5, setting metrics to NaN.
Predicted segmentation is empty for Sample 13, setting metrics to NaN.
True segmentation is empty for Sample 19, setting metrics to NaN.
Predicted segmentation is empty for Sample 25, setting metrics to NaN.
           Sample            Dice Coefficient  \
0    Med_Sample 0 0.6367924528301887043824081   
1    Med_Sample 1 0.6359447004608295150873687   
2    Med_Sample 4 0.6180422264875239379833261   
3    Med_Sample 6 0.8000000000000000444089210   
4    Med_Sample 7 0.5789473684210526549520637   
5    Med_Sample 8 0.0129870129870129878690843   
6    Med_Sample 9 0.7022900763358778108980118   
7   Med_Sample 10 0.6077348066298342565971780   
8   Med_Sample 11 0.1538461538461538546940233   
9   Med_Sample 12 0.7708333333333333703407675   
10  Med_Sample 14 0.5473684210526316151756987   

# Medpy 30-59

In [ ]:
# Medpy 30-59

import os
import numpy as np
import pandas as pd
import nibabel as nib
import torch
from torch.utils.data import Dataset
from medpy.metric.binary import hd, dc, precision as med_precision, recall as med_recall

np.set_printoptions(precision=25)
pd.options.display.float_format = '{:.25f}'.format

# SegmentationDataset Class (PyTorch)
class SegmentationDataset(Dataset):
    def __init__(self, true_seg_dir, pred_seg_dir, transform=None):
        self.true_seg_dir = true_seg_dir
        self.pred_seg_dir = pred_seg_dir
        self.transform = transform

        # Get all files in directories
        self.true_seg_files = sorted(os.listdir(true_seg_dir))
        self.pred_seg_files = sorted(os.listdir(pred_seg_dir))

        # Extract the common parts (e.g. #LIDC-IDRI#LIDC-IDRI-0001#VISIT 1#.nii.gz)
        self.true_seg_base_names = [self.extract_base_name(file) for file in self.true_seg_files]
        self.pred_seg_base_names = [self.extract_base_name(file) for file in self.pred_seg_files]

        # Ensure that there are matching filenames between true and predicted
        assert set(self.true_seg_base_names) == set(self.pred_seg_base_names), \
            "True and predicted segmentation filenames do not match."

    def __len__(self):
        return len(self.true_seg_files)

    def __getitem__(self, idx):
        base_name = self.true_seg_base_names[idx]
        true_seg_filename = [file for file in self.true_seg_files if self.extract_base_name(file) == base_name][0]
        pred_seg_filename = [file for file in self.pred_seg_files if self.extract_base_name(file) == base_name][0]

        true_seg_path = os.path.join(self.true_seg_dir, true_seg_filename)
        pred_seg_path = os.path.join(self.pred_seg_dir, pred_seg_filename)

        true_seg_image = nib.load(true_seg_path).get_fdata()
        pred_seg_image = nib.load(pred_seg_path).get_fdata()

        # Ensure segmentation masks are binary (0 or 1)
        true_seg_image = np.expand_dims(np.clip(true_seg_image, 0, 1), axis=0)
        pred_seg_image = np.expand_dims(np.clip(pred_seg_image, 0, 1), axis=0)

        sample = {'true_seg': true_seg_image, 'pred_seg': pred_seg_image}

        if self.transform:
            sample = self.transform(sample)

        return sample

    def extract_base_name(self, file_name):
        base_name = file_name.split('#', 1)[1]
        return base_name

class ToTensorAndResize(object):
    def __call__(self, sample):
        true_seg = torch.from_numpy(sample['true_seg']).float()
        pred_seg = torch.from_numpy(sample['pred_seg']).float()

        # Reshape the tensors to target size of 128x128x128 (D, H, W)
        true_seg_resized = self.resize_3d(true_seg, (128, 128, 128))
        pred_seg_resized = self.resize_3d(pred_seg, (128, 128, 128))

        return {'true_seg': true_seg_resized, 'pred_seg': pred_seg_resized}

    def resize_3d(self, volume, target_shape):
        """
        Resize a 3D PyTorch tensor to a target shape using trilinear interpolation.
        :param volume: Input 3D PyTorch tensor
        :param target_shape: Desired shape (D, H, W)
        :return: Resized 3D PyTorch tensor
        """
        # Ensure the tensor has at least 3 spatial dimensions
        if len(volume.shape) == 3:  # Shape [D, H, W]
            volume = volume.unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions [1, 1, D, H, W]
        elif len(volume.shape) == 4:  # Shape [C, D, H, W]
            volume = volume.unsqueeze(0)  # Add batch dimension

        # Check the shape before interpolation to make sure it's valid
        if volume.dim() != 5:
            raise ValueError(f"Expected a 5D tensor, but got {volume.dim()}D tensor with shape {volume.shape}")

        # Perform the interpolation (resizing)
        resized_volume = torch.nn.functional.interpolate(volume, size=target_shape, mode='trilinear', align_corners=False)

        # Remove the batch and channel dimensions
        return resized_volume.squeeze(0).squeeze(0)  # Back to shape [D, H, W]

# Load data in PyTorch
true_segmentations_dir = '/content/drive/MyDrive/Mr-Alizadeh/Train-760/CT-segmentation'
predicted_masks_dir = '/content/drive/MyDrive/Mr-Alizadeh/Reconnet_predicted_masks'
# Create dataset (PyTorch)
dataset = SegmentationDataset(true_seg_dir=true_segmentations_dir, pred_seg_dir=predicted_masks_dir, transform=ToTensorAndResize())

# Initialize the results dictionary
results = {}

# Iterate through the dataset and calculate metrics using Scikit-learn, SimpleITK, and MedPy
for i in range(30,60):
    sample = dataset[i]

    # Convert PyTorch tensors to NumPy arrays
    y_true_np = sample['true_seg'].numpy().astype(np.int32).flatten()
    y_pred_np = sample['pred_seg'].numpy().astype(np.int32).flatten()
        # بررسی اینکه تصاویر واقعی یا پیش‌بینی‌شده خالی نیستند
    if np.count_nonzero(y_true_np) == 0:
        print(f"True segmentation is empty for Sample {i}, setting metrics to NaN.")
        sklearn_accuracy = np.nan
        sklearn_precision = np.nan
        sklearn_recall = np.nan
        # سایر متریک‌ها را با NaN پر کنید...
        continue

    if np.count_nonzero(y_pred_np) == 0:
        print(f"Predicted segmentation is empty for Sample {i}, setting metrics to NaN.")
        sklearn_accuracy = np.nan
        sklearn_precision = np.nan
        sklearn_recall = np.nan
        # سایر متریک‌ها را با NaN پر کنید...
        continue
    # بررسی اینکه تصاویر خالی نیستند
    if np.count_nonzero(y_true_np) == 0:
        raise ValueError(f"True segmentation is empty for Sample {i}")

    if np.count_nonzero(y_pred_np) == 0:
        raise ValueError(f"Predicted segmentation is empty for Sample {i}")

    # بررسی تطابق ابعاد تصاویر
    if sample['true_seg'].shape != sample['pred_seg'].shape:
        raise ValueError(f"Shape mismatch: True segmentation shape {sample['true_seg'].shape}, "
                         f"Predicted segmentation shape {sample['pred_seg'].shape}")


    # Metrics using MedPy
    dc_medpy = dc(sample['pred_seg'].numpy().astype(np.int32).flatten(), sample['true_seg'].numpy().astype(np.int32).flatten())
    hausdorff_distance_med = hd(sample['pred_seg'].numpy().astype(np.int32).flatten(), sample['true_seg'].numpy().astype(np.int32).flatten())
    precision_medpy = med_precision(sample['pred_seg'].numpy().astype(np.int32).flatten(), sample['true_seg'].numpy().astype(np.int32).flatten())
    recall_medpy = med_recall(sample['pred_seg'].numpy().astype(np.int32).flatten(), sample['true_seg'].numpy().astype(np.int32).flatten())

    # Manually calculate IoU for MedPy (Jaccard Index)
    tp = np.sum((y_true_np == 1) & (y_pred_np == 1))
    fp = np.sum((y_true_np == 0) & (y_pred_np == 1))
    fn = np.sum((y_true_np == 1) & (y_pred_np == 0))
    iou_medpy = tp / (tp + fp + fn + 1e-6)  # Add a small epsilon to avoid division by zero

    # Calculate F1 Score for MedPy
    f1_medpy = 2 * (precision_medpy * recall_medpy) / (precision_medpy + recall_medpy + 1e-6)

    # Store the metrics
    results[f'Med_Sample {i}'] = {
            'Dice Coefficient': dc_medpy,
            'Hausdorff Distance': hausdorff_distance_med,
            'IoU': iou_medpy,
            'Precision': precision_medpy,
            'Recall': recall_medpy,
            'F1 Score': f1_medpy
    }

# Convert results to a DataFrame for better readability
results_df = pd.DataFrame(results).T
results_df.index.name = 'Sample'
results_df.reset_index(inplace=True)
print(results_df)

# Save the DataFrame to an Excel file
results_df.to_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Segmentation/binary_Med_30-59.xlsx', index=False)
# Medpy 30-59

Predicted segmentation is empty for Sample 38, setting metrics to NaN.
True segmentation is empty for Sample 50, setting metrics to NaN.
           Sample            Dice Coefficient  \
0   Med_Sample 30 0.6093750000000000000000000   
1   Med_Sample 31 0.5454545454545454141737082   
2   Med_Sample 32 0.8487804878048780254928829   
3   Med_Sample 33 0.5937500000000000000000000   
4   Med_Sample 34 0.6153846153846154187760931   
5   Med_Sample 35 0.7843866171003717191467786   
6   Med_Sample 36 0.7441860465116278966490881   
7   Med_Sample 37 0.3800000000000000044408921   
8   Med_Sample 39 0.7588932806324110158513463   
9   Med_Sample 40 0.6666666666666666296592325   
10  Med_Sample 41 0.9023090586145647851168405   
11  Med_Sample 42 0.5561694290976059162900924   
12  Med_Sample 43 0.6939611054247697419228302   
13  Med_Sample 44 0.5365853658536585690796983   
14  Med_Sample 45 0.7742857142857142438430174   
15  Med_Sample 46 0.0000000000000000000000000   
16  Med_Sample 47 0.1818181818

# Medpy 60-99

In [ ]:
# Medpy 60-99

import os
import numpy as np
import pandas as pd
import nibabel as nib
import torch
from torch.utils.data import Dataset
from medpy.metric.binary import hd, dc, precision as med_precision, recall as med_recall

np.set_printoptions(precision=25)
pd.options.display.float_format = '{:.25f}'.format

# SegmentationDataset Class (PyTorch)
class SegmentationDataset(Dataset):
    def __init__(self, true_seg_dir, pred_seg_dir, transform=None):
        self.true_seg_dir = true_seg_dir
        self.pred_seg_dir = pred_seg_dir
        self.transform = transform

        # Get all files in directories
        self.true_seg_files = sorted(os.listdir(true_seg_dir))
        self.pred_seg_files = sorted(os.listdir(pred_seg_dir))

        # Extract the common parts (e.g. #LIDC-IDRI#LIDC-IDRI-0001#VISIT 1#.nii.gz)
        self.true_seg_base_names = [self.extract_base_name(file) for file in self.true_seg_files]
        self.pred_seg_base_names = [self.extract_base_name(file) for file in self.pred_seg_files]

        # Ensure that there are matching filenames between true and predicted
        assert set(self.true_seg_base_names) == set(self.pred_seg_base_names), \
            "True and predicted segmentation filenames do not match."

    def __len__(self):
        return len(self.true_seg_files)

    def __getitem__(self, idx):
        base_name = self.true_seg_base_names[idx]
        true_seg_filename = [file for file in self.true_seg_files if self.extract_base_name(file) == base_name][0]
        pred_seg_filename = [file for file in self.pred_seg_files if self.extract_base_name(file) == base_name][0]

        true_seg_path = os.path.join(self.true_seg_dir, true_seg_filename)
        pred_seg_path = os.path.join(self.pred_seg_dir, pred_seg_filename)

        true_seg_image = nib.load(true_seg_path).get_fdata()
        pred_seg_image = nib.load(pred_seg_path).get_fdata()

        # Ensure segmentation masks are binary (0 or 1)
        true_seg_image = np.expand_dims(np.clip(true_seg_image, 0, 1), axis=0)
        pred_seg_image = np.expand_dims(np.clip(pred_seg_image, 0, 1), axis=0)

        sample = {'true_seg': true_seg_image, 'pred_seg': pred_seg_image}

        if self.transform:
            sample = self.transform(sample)

        return sample

    def extract_base_name(self, file_name):
        base_name = file_name.split('#', 1)[1]
        return base_name

class ToTensorAndResize(object):
    def __call__(self, sample):
        true_seg = torch.from_numpy(sample['true_seg']).float()
        pred_seg = torch.from_numpy(sample['pred_seg']).float()

        # Reshape the tensors to target size of 128x128x128 (D, H, W)
        true_seg_resized = self.resize_3d(true_seg, (128, 128, 128))
        pred_seg_resized = self.resize_3d(pred_seg, (128, 128, 128))

        return {'true_seg': true_seg_resized, 'pred_seg': pred_seg_resized}

    def resize_3d(self, volume, target_shape):
        """
        Resize a 3D PyTorch tensor to a target shape using trilinear interpolation.
        :param volume: Input 3D PyTorch tensor
        :param target_shape: Desired shape (D, H, W)
        :return: Resized 3D PyTorch tensor
        """
        # Ensure the tensor has at least 3 spatial dimensions
        if len(volume.shape) == 3:  # Shape [D, H, W]
            volume = volume.unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions [1, 1, D, H, W]
        elif len(volume.shape) == 4:  # Shape [C, D, H, W]
            volume = volume.unsqueeze(0)  # Add batch dimension

        # Check the shape before interpolation to make sure it's valid
        if volume.dim() != 5:
            raise ValueError(f"Expected a 5D tensor, but got {volume.dim()}D tensor with shape {volume.shape}")

        # Perform the interpolation (resizing)
        resized_volume = torch.nn.functional.interpolate(volume, size=target_shape, mode='trilinear', align_corners=False)

        # Remove the batch and channel dimensions
        return resized_volume.squeeze(0).squeeze(0)  # Back to shape [D, H, W]

# Load data in PyTorch
true_segmentations_dir = '/content/drive/MyDrive/Mr-Alizadeh/Train-760/CT-segmentation'
predicted_masks_dir = '/content/drive/MyDrive/Mr-Alizadeh/Reconnet_predicted_masks'
# Create dataset (PyTorch)
dataset = SegmentationDataset(true_seg_dir=true_segmentations_dir, pred_seg_dir=predicted_masks_dir, transform=ToTensorAndResize())

# Initialize the results dictionary
results = {}

# Iterate through the dataset and calculate metrics using Scikit-learn, SimpleITK, and MedPy
for i in range(60,100):
    sample = dataset[i]

    # Convert PyTorch tensors to NumPy arrays
    y_true_np = sample['true_seg'].numpy().astype(np.int32).flatten()
    y_pred_np = sample['pred_seg'].numpy().astype(np.int32).flatten()
        # بررسی اینکه تصاویر واقعی یا پیش‌بینی‌شده خالی نیستند
    if np.count_nonzero(y_true_np) == 0:
        print(f"True segmentation is empty for Sample {i}, setting metrics to NaN.")
        sklearn_accuracy = np.nan
        sklearn_precision = np.nan
        sklearn_recall = np.nan
        # سایر متریک‌ها را با NaN پر کنید...
        continue

    if np.count_nonzero(y_pred_np) == 0:
        print(f"Predicted segmentation is empty for Sample {i}, setting metrics to NaN.")
        sklearn_accuracy = np.nan
        sklearn_precision = np.nan
        sklearn_recall = np.nan
        # سایر متریک‌ها را با NaN پر کنید...
        continue
    # بررسی اینکه تصاویر خالی نیستند
    if np.count_nonzero(y_true_np) == 0:
        raise ValueError(f"True segmentation is empty for Sample {i}")

    if np.count_nonzero(y_pred_np) == 0:
        raise ValueError(f"Predicted segmentation is empty for Sample {i}")

    # بررسی تطابق ابعاد تصاویر
    if sample['true_seg'].shape != sample['pred_seg'].shape:
        raise ValueError(f"Shape mismatch: True segmentation shape {sample['true_seg'].shape}, "
                         f"Predicted segmentation shape {sample['pred_seg'].shape}")


    # Metrics using MedPy
    dc_medpy = dc(sample['pred_seg'].numpy().astype(np.int32).flatten(), sample['true_seg'].numpy().astype(np.int32).flatten())
    hausdorff_distance_med = hd(sample['pred_seg'].numpy().astype(np.int32).flatten(), sample['true_seg'].numpy().astype(np.int32).flatten())
    precision_medpy = med_precision(sample['pred_seg'].numpy().astype(np.int32).flatten(), sample['true_seg'].numpy().astype(np.int32).flatten())
    recall_medpy = med_recall(sample['pred_seg'].numpy().astype(np.int32).flatten(), sample['true_seg'].numpy().astype(np.int32).flatten())

    # Manually calculate IoU for MedPy (Jaccard Index)
    tp = np.sum((y_true_np == 1) & (y_pred_np == 1))
    fp = np.sum((y_true_np == 0) & (y_pred_np == 1))
    fn = np.sum((y_true_np == 1) & (y_pred_np == 0))
    iou_medpy = tp / (tp + fp + fn + 1e-6)  # Add a small epsilon to avoid division by zero

    # Calculate F1 Score for MedPy
    f1_medpy = 2 * (precision_medpy * recall_medpy) / (precision_medpy + recall_medpy + 1e-6)

    # Store the metrics
    results[f'Med_Sample {i}'] = {
            'Dice Coefficient': dc_medpy,
            'Hausdorff Distance': hausdorff_distance_med,
            'IoU': iou_medpy,
            'Precision': precision_medpy,
            'Recall': recall_medpy,
            'F1 Score': f1_medpy
    }

# Convert results to a DataFrame for better readability
results_df = pd.DataFrame(results).T
results_df.index.name = 'Sample'
results_df.reset_index(inplace=True)
print(results_df)

# Save the DataFrame to an Excel file
results_df.to_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Segmentation/binary_Med_60-99.xlsx', index=False)
# Medpy 60-99

Predicted segmentation is empty for Sample 63, setting metrics to NaN.
Predicted segmentation is empty for Sample 69, setting metrics to NaN.
True segmentation is empty for Sample 72, setting metrics to NaN.
Predicted segmentation is empty for Sample 74, setting metrics to NaN.
Predicted segmentation is empty for Sample 75, setting metrics to NaN.
True segmentation is empty for Sample 77, setting metrics to NaN.
Predicted segmentation is empty for Sample 85, setting metrics to NaN.
Predicted segmentation is empty for Sample 96, setting metrics to NaN.
Predicted segmentation is empty for Sample 97, setting metrics to NaN.
           Sample            Dice Coefficient  \
0   Med_Sample 60 0.6611570247933884481028599   
1   Med_Sample 61 0.8529411764705882026404993   
2   Med_Sample 62 0.2000000000000000111022302   
3   Med_Sample 64 0.6598360655737705027235052   
4   Med_Sample 65 0.5809128630705394646938089   
5   Med_Sample 66 0.8690807799442896719455121   
6   Med_Sample 67 0.57142857

# 2D binary Medpy 2 pic

In [ ]:
# 2D Medpy 2 pic

import os
import numpy as np
import pandas as pd
import nibabel as nib
import torch
from torch.utils.data import Dataset
from medpy.metric.binary import hd, dc, precision as med_precision, recall as med_recall

np.set_printoptions(precision=25)
pd.options.display.float_format = '{:.25f}'.format

# SegmentationDataset Class (PyTorch)
class SegmentationDataset(Dataset):
    def __init__(self, true_seg_dir, pred_seg_dir, transform=None):
        self.true_seg_dir = true_seg_dir
        self.pred_seg_dir = pred_seg_dir
        self.transform = transform

        # Get all files in directories
        self.true_seg_files = sorted(os.listdir(true_seg_dir))
        self.pred_seg_files = sorted(os.listdir(pred_seg_dir))

        # Extract the common parts (e.g. #LIDC-IDRI#LIDC-IDRI-0001#VISIT 1#.nii.gz)
        self.true_seg_base_names = [self.extract_base_name(file) for file in self.true_seg_files]
        self.pred_seg_base_names = [self.extract_base_name(file) for file in self.pred_seg_files]

        # Ensure that there are matching filenames between true and predicted
        assert set(self.true_seg_base_names) == set(self.pred_seg_base_names), \
            "True and predicted segmentation filenames do not match."

    def __len__(self):
        return len(self.true_seg_files)

    def __getitem__(self, idx):
        base_name = self.true_seg_base_names[idx]
        true_seg_filename = [file for file in self.true_seg_files if self.extract_base_name(file) == base_name][0]
        pred_seg_filename = [file for file in self.pred_seg_files if self.extract_base_name(file) == base_name][0]

        true_seg_path = os.path.join(self.true_seg_dir, true_seg_filename)
        pred_seg_path = os.path.join(self.pred_seg_dir, pred_seg_filename)

        true_seg_image = nib.load(true_seg_path).get_fdata()
        pred_seg_image = nib.load(pred_seg_path).get_fdata()

        # Ensure segmentation masks are binary (0 or 1)
        true_seg_image = np.expand_dims(np.clip(true_seg_image, 0, 1), axis=0)
        pred_seg_image = np.expand_dims(np.clip(pred_seg_image, 0, 1), axis=0)

        sample = {'true_seg': true_seg_image, 'pred_seg': pred_seg_image}

        if self.transform:
            sample = self.transform(sample)

        return sample

    def extract_base_name(self, file_name):
        base_name = file_name.split('#', 1)[1]
        return base_name

class ToTensorAndResize(object):
    def __call__(self, sample):
        true_seg = torch.from_numpy(sample['true_seg']).float()
        pred_seg = torch.from_numpy(sample['pred_seg']).float()

        # Reshape the tensors to target size of 128x128x128 (D, H, W)
        true_seg_resized = self.resize_3d(true_seg, (128, 128, 128))
        pred_seg_resized = self.resize_3d(pred_seg, (128, 128, 128))

        return {'true_seg': true_seg_resized, 'pred_seg': pred_seg_resized}

    def resize_3d(self, volume, target_shape):
        """
        Resize a 3D PyTorch tensor to a target shape using trilinear interpolation.
        :param volume: Input 3D PyTorch tensor
        :param target_shape: Desired shape (D, H, W)
        :return: Resized 3D PyTorch tensor
        """
        # Ensure the tensor has at least 3 spatial dimensions
        if len(volume.shape) == 3:  # Shape [D, H, W]
            volume = volume.unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions [1, 1, D, H, W]
        elif len(volume.shape) == 4:  # Shape [C, D, H, W]
            volume = volume.unsqueeze(0)  # Add batch dimension

        # Check the shape before interpolation to make sure it's valid
        if volume.dim() != 5:
            raise ValueError(f"Expected a 5D tensor, but got {volume.dim()}D tensor with shape {volume.shape}")

        # Perform the interpolation (resizing)
        resized_volume = torch.nn.functional.interpolate(volume, size=target_shape, mode='trilinear', align_corners=False)

        # Remove the batch and channel dimensions
        return resized_volume.squeeze(0).squeeze(0)  # Back to shape [D, H, W]

# Load data in PyTorch
true_segmentations_dir = '/content/drive/MyDrive/Saba/true_seg'
predicted_masks_dir = '/content/drive/MyDrive/Saba/pred_seg'

# Create dataset (PyTorch)
dataset = SegmentationDataset(true_seg_dir=true_segmentations_dir, pred_seg_dir=predicted_masks_dir, transform=ToTensorAndResize())

# Initialize the results dictionary
results = {}

# Iterate through the dataset and calculate metrics using Scikit-learn, SimpleITK, and MedPy
for i in range(len(dataset)):
    sample = dataset[i]

    # Convert PyTorch tensors to NumPy arrays
    y_true_np = sample['true_seg'].numpy().astype(np.int32).flatten()
    y_pred_np = sample['pred_seg'].numpy().astype(np.int32).flatten()
        # بررسی اینکه تصاویر واقعی یا پیش‌بینی‌شده خالی نیستند
    if np.count_nonzero(y_true_np) == 0:
        print(f"True segmentation is empty for Sample {i}, setting metrics to NaN.")
        sklearn_accuracy = np.nan
        sklearn_precision = np.nan
        sklearn_recall = np.nan
        # سایر متریک‌ها را با NaN پر کنید...
        continue

    if np.count_nonzero(y_pred_np) == 0:
        print(f"Predicted segmentation is empty for Sample {i}, setting metrics to NaN.")
        sklearn_accuracy = np.nan
        sklearn_precision = np.nan
        sklearn_recall = np.nan
        # سایر متریک‌ها را با NaN پر کنید...
        continue
    # بررسی اینکه تصاویر خالی نیستند
    if np.count_nonzero(y_true_np) == 0:
        raise ValueError(f"True segmentation is empty for Sample {i}")

    if np.count_nonzero(y_pred_np) == 0:
        raise ValueError(f"Predicted segmentation is empty for Sample {i}")

    # بررسی تطابق ابعاد تصاویر
    if sample['true_seg'].shape != sample['pred_seg'].shape:
        raise ValueError(f"Shape mismatch: True segmentation shape {sample['true_seg'].shape}, "
                         f"Predicted segmentation shape {sample['pred_seg'].shape}")


    # Metrics using MedPy
    dc_medpy = dc(sample['pred_seg'].numpy().astype(np.int32).flatten(), sample['true_seg'].numpy().astype(np.int32).flatten())
    hausdorff_distance_med = hd(sample['pred_seg'].numpy().astype(np.int32).flatten(), sample['true_seg'].numpy().astype(np.int32).flatten())
    precision_medpy = med_precision(sample['pred_seg'].numpy().astype(np.int32).flatten(), sample['true_seg'].numpy().astype(np.int32).flatten())
    recall_medpy = med_recall(sample['pred_seg'].numpy().astype(np.int32).flatten(), sample['true_seg'].numpy().astype(np.int32).flatten())

    # Manually calculate IoU for MedPy (Jaccard Index)
    tp = np.sum((y_true_np == 1) & (y_pred_np == 1))
    fp = np.sum((y_true_np == 0) & (y_pred_np == 1))
    fn = np.sum((y_true_np == 1) & (y_pred_np == 0))
    iou_medpy = tp / (tp + fp + fn + 1e-6)  # Add a small epsilon to avoid division by zero

    # Calculate F1 Score for MedPy
    f1_medpy = 2 * (precision_medpy * recall_medpy) / (precision_medpy + recall_medpy + 1e-6)

    # Store the metrics
    results[f'Med_Sample {i}'] = {
            'Dice Coefficient': dc_medpy,
            'Hausdorff Distance': hausdorff_distance_med,
            'IoU': iou_medpy,
            'Precision': precision_medpy,
            'Recall': recall_medpy,
            'F1 Score': f1_medpy
    }

# Convert results to a DataFrame for better readability
results_df = pd.DataFrame(results).T
results_df.index.name = 'Sample'
results_df.reset_index(inplace=True)
print(results_df)

# Save the DataFrame to an Excel file
results_df.to_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Segmentation/2D_binary_Med_2pic.xlsx', index=False)
# 2D Medpy 2 pic

         Sample            Dice Coefficient               Hausdorff Distance  \
0  Med_Sample 0 0.6367924528301887043824081  16256.0000000000000000000000000   
1  Med_Sample 1 0.6359447004608295150873687 360855.0000000000000000000000000   

                          IoU                   Precision  \
0 0.4671280260653009608340369 0.4671280276816608956202970   
1 0.4662162130661066661296843 0.4893617021276595591139369   

                       Recall                    F1 Score  
0 1.0000000000000000000000000 0.6367920187903456374911570  
1 0.9078947368421053099041274 0.6359442453230126357510699  


# SL 760

In [ ]:
# SL 760

import os
import numpy as np
import pandas as pd
import nibabel as nib
import torch
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, jaccard_score, cohen_kappa_score
import SimpleITK as sitk
from medpy.metric.binary import hd, dc, precision as med_precision, recall as med_recall

np.set_printoptions(precision=25)
pd.options.display.float_format = '{:.25f}'.format

# SegmentationDataset Class (PyTorch)
class SegmentationDataset(Dataset):
    def __init__(self, true_seg_dir, pred_seg_dir, transform=None):
        self.true_seg_dir = true_seg_dir
        self.pred_seg_dir = pred_seg_dir
        self.transform = transform

        # Get all files in directories
        self.true_seg_files = sorted(os.listdir(true_seg_dir))
        self.pred_seg_files = sorted(os.listdir(pred_seg_dir))

        # Extract the common parts (e.g. #LIDC-IDRI#LIDC-IDRI-0001#VISIT 1#.nii.gz)
        self.true_seg_base_names = [self.extract_base_name(file) for file in self.true_seg_files]
        self.pred_seg_base_names = [self.extract_base_name(file) for file in self.pred_seg_files]

        # Ensure that there are matching filenames between true and predicted
        assert set(self.true_seg_base_names) == set(self.pred_seg_base_names), \
            "True and predicted segmentation filenames do not match."

    def __len__(self):
        return len(self.true_seg_files)

    def __getitem__(self, idx):
        base_name = self.true_seg_base_names[idx]
        true_seg_filename = [file for file in self.true_seg_files if self.extract_base_name(file) == base_name][0]
        pred_seg_filename = [file for file in self.pred_seg_files if self.extract_base_name(file) == base_name][0]

        true_seg_path = os.path.join(self.true_seg_dir, true_seg_filename)
        pred_seg_path = os.path.join(self.pred_seg_dir, pred_seg_filename)

        true_seg_image = nib.load(true_seg_path).get_fdata()
        pred_seg_image = nib.load(pred_seg_path).get_fdata()

        # Ensure segmentation masks are binary (0 or 1)
        true_seg_image = np.expand_dims(np.clip(true_seg_image, 0, 1), axis=0)
        pred_seg_image = np.expand_dims(np.clip(pred_seg_image, 0, 1), axis=0)

        sample = {'true_seg': true_seg_image, 'pred_seg': pred_seg_image}

        if self.transform:
            sample = self.transform(sample)

        return sample

    def extract_base_name(self, file_name):
        base_name = file_name.split('#', 1)[1]
        return base_name

class ToTensorAndResize(object):
    def __call__(self, sample):
        true_seg = torch.from_numpy(sample['true_seg']).float()
        pred_seg = torch.from_numpy(sample['pred_seg']).float()

        # Reshape the tensors to target size of 128x128x128 (D, H, W)
        true_seg_resized = self.resize_3d(true_seg, (128, 128, 128))
        pred_seg_resized = self.resize_3d(pred_seg, (128, 128, 128))

        return {'true_seg': true_seg_resized, 'pred_seg': pred_seg_resized}

    def resize_3d(self, volume, target_shape):
        """
        Resize a 3D PyTorch tensor to a target shape using trilinear interpolation.
        :param volume: Input 3D PyTorch tensor
        :param target_shape: Desired shape (D, H, W)
        :return: Resized 3D PyTorch tensor
        """
        # Ensure the tensor has at least 3 spatial dimensions
        if len(volume.shape) == 3:  # Shape [D, H, W]
            volume = volume.unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions [1, 1, D, H, W]
        elif len(volume.shape) == 4:  # Shape [C, D, H, W]
            volume = volume.unsqueeze(0)  # Add batch dimension

        # Check the shape before interpolation to make sure it's valid
        if volume.dim() != 5:
            raise ValueError(f"Expected a 5D tensor, but got {volume.dim()}D tensor with shape {volume.shape}")

        # Perform the interpolation (resizing)
        resized_volume = torch.nn.functional.interpolate(volume, size=target_shape, mode='trilinear', align_corners=False)

        # Remove the batch and channel dimensions
        return resized_volume.squeeze(0).squeeze(0)  # Back to shape [D, H, W]

# Load data in PyTorch
true_segmentations_dir = '/content/drive/MyDrive/Mr-Alizadeh/Train-760/CT-segmentation'
predicted_masks_dir = '/content/drive/MyDrive/Mr-Alizadeh/Reconnet_predicted_masks'

# Create dataset (PyTorch)
dataset = SegmentationDataset(true_seg_dir=true_segmentations_dir, pred_seg_dir=predicted_masks_dir, transform=ToTensorAndResize())

# Initialize the results dictionary
results = {}

# Iterate through the dataset and calculate metrics using Scikit-learn, SimpleITK, and MedPy
for i in range(len(dataset)):
    sample = dataset[i]

    # Convert PyTorch tensors to NumPy arrays
    y_true_np = sample['true_seg'].numpy().astype(np.int32).flatten()
    y_pred_np = sample['pred_seg'].numpy().astype(np.int32).flatten()
        # بررسی اینکه تصاویر واقعی یا پیش‌بینی‌شده خالی نیستند
    if np.count_nonzero(y_true_np) == 0:
        print(f"True segmentation is empty for Sample {i}, setting metrics to NaN.")
        sklearn_accuracy = np.nan
        sklearn_precision = np.nan
        sklearn_recall = np.nan
        # سایر متریک‌ها را با NaN پر کنید...
        continue

    if np.count_nonzero(y_pred_np) == 0:
        print(f"Predicted segmentation is empty for Sample {i}, setting metrics to NaN.")
        sklearn_accuracy = np.nan
        sklearn_precision = np.nan
        sklearn_recall = np.nan
        # سایر متریک‌ها را با NaN پر کنید...
        continue
    # بررسی اینکه تصاویر خالی نیستند
    if np.count_nonzero(y_true_np) == 0:
        raise ValueError(f"True segmentation is empty for Sample {i}")

    if np.count_nonzero(y_pred_np) == 0:
        raise ValueError(f"Predicted segmentation is empty for Sample {i}")

    # بررسی تطابق ابعاد تصاویر
    if sample['true_seg'].shape != sample['pred_seg'].shape:
        raise ValueError(f"Shape mismatch: True segmentation shape {sample['true_seg'].shape}, "
                         f"Predicted segmentation shape {sample['pred_seg'].shape}")

    # Metrics using Scikit-learn
    sklearn_accuracy = accuracy_score(y_true_np, y_pred_np)
    sklearn_precision = precision_score(y_true_np, y_pred_np)
    sklearn_recall = recall_score(y_true_np, y_pred_np)
    sklearn_f1 = f1_score(y_true_np, y_pred_np)
    sklearn_jaccard = jaccard_score(y_true_np, y_pred_np)
    sklearn_kappa = cohen_kappa_score(y_true_np, y_pred_np)

    # Boundary F1 Score (calculated manually or using Scikit-learn's precision/recall)
    bf_score_skl = 2 * (sklearn_precision * sklearn_recall) / (sklearn_precision + sklearn_recall + 1e-6)


    # Store the metrics
    results[f'Sk_Sample {i}'] = {
            'Accuracy': sklearn_accuracy,
            'Precision': sklearn_precision,
            'Recall': sklearn_recall,
            'F1 Score': sklearn_f1,
            'IoU': sklearn_jaccard,
            'Kappa': sklearn_kappa,
            'BF Score': bf_score_skl
        }
# Convert results to a DataFrame for better readability
results_df = pd.DataFrame(results).T
results_df.index.name = 'Sample'
results_df.reset_index(inplace=True)
print(results_df)

# Save the DataFrame to an Excel file
results_df.to_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Segmentation/binary_SL_0-29.xlsx', index=False)


Predicted segmentation is empty for Sample 2, setting metrics to NaN.
Predicted segmentation is empty for Sample 3, setting metrics to NaN.
True segmentation is empty for Sample 5, setting metrics to NaN.
Predicted segmentation is empty for Sample 13, setting metrics to NaN.
True segmentation is empty for Sample 19, setting metrics to NaN.
Predicted segmentation is empty for Sample 25, setting metrics to NaN.
Predicted segmentation is empty for Sample 38, setting metrics to NaN.
True segmentation is empty for Sample 50, setting metrics to NaN.
Predicted segmentation is empty for Sample 63, setting metrics to NaN.
Predicted segmentation is empty for Sample 69, setting metrics to NaN.
True segmentation is empty for Sample 72, setting metrics to NaN.
Predicted segmentation is empty for Sample 74, setting metrics to NaN.
Predicted segmentation is empty for Sample 75, setting metrics to NaN.
True segmentation is empty for Sample 77, setting metrics to NaN.
Predicted segmentation is empty fo

# TF 15 pic

In [ ]:
# TF 10 pic

import os
import numpy as np
import pandas as pd
import nibabel as nib
import torch
import tensorflow as tf
from torch.utils.data import Dataset
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy, MeanIoU

np.set_printoptions(precision=25)
pd.options.display.float_format = '{:.25f}'.format

# SegmentationDataset Class (PyTorch)
class SegmentationDataset(Dataset):
    def __init__(self, true_seg_dir, pred_seg_dir, transform=None):
        self.true_seg_dir = true_seg_dir
        self.pred_seg_dir = pred_seg_dir
        self.transform = transform

        # Get all files in directories
        self.true_seg_files = sorted(os.listdir(true_seg_dir))
        self.pred_seg_files = sorted(os.listdir(pred_seg_dir))

        # Extract the common parts (e.g. #LIDC-IDRI#LIDC-IDRI-0001#VISIT 1#.nii.gz)
        self.true_seg_base_names = [self.extract_base_name(file) for file in self.true_seg_files]
        self.pred_seg_base_names = [self.extract_base_name(file) for file in self.pred_seg_files]

        # Ensure that there are matching filenames between true and predicted
        assert set(self.true_seg_base_names) == set(self.pred_seg_base_names), \
            "True and predicted segmentation filenames do not match."

    def __len__(self):
        return len(self.true_seg_files)

    def __getitem__(self, idx):
        base_name = self.true_seg_base_names[idx]
        true_seg_filename = [file for file in self.true_seg_files if self.extract_base_name(file) == base_name][0]
        pred_seg_filename = [file for file in self.pred_seg_files if self.extract_base_name(file) == base_name][0]

        true_seg_path = os.path.join(self.true_seg_dir, true_seg_filename)
        pred_seg_path = os.path.join(self.pred_seg_dir, pred_seg_filename)

        true_seg_image = nib.load(true_seg_path).get_fdata()
        pred_seg_image = nib.load(pred_seg_path).get_fdata()

        # Ensure segmentation masks are binary (0 or 1)
        true_seg_image = np.expand_dims(np.clip(true_seg_image, 0, 1), axis=0)
        pred_seg_image = np.expand_dims(np.clip(pred_seg_image, 0, 1), axis=0)

        sample = {'true_seg': true_seg_image, 'pred_seg': pred_seg_image}

        if self.transform:
            sample = self.transform(sample)

        return sample

    def extract_base_name(self, file_name):
        base_name = file_name.split('#', 1)[1]
        return base_name

class ToTensorAndResize(object):
    def __call__(self, sample):
        true_seg = torch.from_numpy(sample['true_seg']).float()
        pred_seg = torch.from_numpy(sample['pred_seg']).float()

        # Reshape the tensors to target size of 128x128x128 (D, H, W)
        true_seg_resized = self.resize_3d(true_seg, (128, 128, 128))
        pred_seg_resized = self.resize_3d(pred_seg, (128, 128, 128))

        return {'true_seg': true_seg_resized, 'pred_seg': pred_seg_resized}

    def resize_3d(self, volume, target_shape):
        """
        Resize a 3D PyTorch tensor to a target shape using trilinear interpolation.
        :param volume: Input 3D PyTorch tensor
        :param target_shape: Desired shape (D, H, W)
        :return: Resized 3D PyTorch tensor
        """
        # Ensure the tensor has at least 3 spatial dimensions
        if len(volume.shape) == 3:  # Shape [D, H, W]
            volume = volume.unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions [1, 1, D, H, W]
        elif len(volume.shape) == 4:  # Shape [C, D, H, W]
            volume = volume.unsqueeze(0)  # Add batch dimension

        # Check the shape before interpolation to make sure it's valid
        if volume.dim() != 5:
            raise ValueError(f"Expected a 5D tensor, but got {volume.dim()}D tensor with shape {volume.shape}")

        # Perform the interpolation (resizing)
        resized_volume = torch.nn.functional.interpolate(volume, size=target_shape, mode='trilinear', align_corners=False)

        # Remove the batch and channel dimensions
        return resized_volume.squeeze(0).squeeze(0)  # Back to shape [D, H, W]


# Load data in PyTorch
true_segmentations_dir = '/content/drive/MyDrive/Loss Function Evaluation/Data/Segmentation/true'
predicted_masks_dir = '/content/drive/MyDrive/Loss Function Evaluation/Data/Segmentation/pred'

# Create dataset (PyTorch)
dataset = SegmentationDataset(true_seg_dir=true_segmentations_dir, pred_seg_dir=predicted_masks_dir, transform=ToTensorAndResize())

# Initialize the results dictionary for TensorFlow metrics
results_tf = {}

# Iterate through the dataset and calculate metrics using TensorFlow
for i in range(len(dataset)):
    sample = dataset[i]

    # Convert PyTorch tensors to NumPy and then TensorFlow tensors
    y_true_np = sample['true_seg'].numpy()
    y_pred_np = sample['pred_seg'].numpy()

    y_true_tf = tf.convert_to_tensor(y_true_np, dtype=tf.float32)
    y_pred_tf = tf.convert_to_tensor(y_pred_np, dtype=tf.float32)

    # Flatten the tensors for certain metrics
    y_true_flat = tf.reshape(y_true_tf, [-1])
    y_pred_flat = tf.reshape(y_pred_tf, [-1])

    # Convert to binary format for metrics
    y_true_bin = tf.cast(y_true_flat, dtype=tf.int32)
    y_pred_bin = tf.cast(y_pred_flat, dtype=tf.int32)

    # TensorFlow/Keras metrics
    accuracy = BinaryAccuracy()
    precision = Precision()
    recall = Recall()
    iou = MeanIoU(num_classes=2)

    # Update and compute the metrics
    accuracy.update_state(y_true_bin, y_pred_bin)
    precision.update_state(y_true_bin, y_pred_bin)
    recall.update_state(y_true_bin, y_pred_bin)
    iou.update_state(y_true_bin, y_pred_bin)

    # Calculate boundary F1 and Hausdorff Distance
    def extract_boundaries_tf(mask):
        eroded_mask = tf.nn.max_pool3d(mask[None, None, ...], ksize=3, strides=1, padding='SAME')[0, 0]
        boundary = tf.cast(tf.not_equal(mask, eroded_mask), dtype=tf.float32)
        return boundary

    def boundary_f1_score_tf(y_true, y_pred, dilation_radius=1):
        y_true_boundary = extract_boundaries_tf(y_true)
        y_pred_boundary = extract_boundaries_tf(y_pred)
        y_true_boundary_flat = tf.reshape(y_true_boundary, [-1])
        y_pred_boundary_flat = tf.reshape(y_pred_boundary, [-1])
        precision_value = tf.reduce_sum(y_true_boundary_flat * y_pred_boundary_flat) / (tf.reduce_sum(y_pred_boundary_flat) + 1e-6)
        recall_value = tf.reduce_sum(y_true_boundary_flat * y_pred_boundary_flat) / (tf.reduce_sum(y_true_boundary_flat) + 1e-6)
        bf_score = 2 * (precision_value * recall_value) / (precision_value + recall_value + 1e-6)
        return bf_score.numpy()

    def hausdorff_distance_tf(y_true, y_pred):
        """
        Calculate the Hausdorff Distance between two 3D binary masks.
        :param y_true: Ground truth mask
        :param y_pred: Predicted mask
        :return: Hausdorff Distance
        """
        # Get the indices of the true and predicted points
        y_true_points = tf.where(tf.equal(y_true, 1))
        y_pred_points = tf.where(tf.equal(y_pred, 1))

        # Cast the points to float32 for distance calculation
        y_true_points = tf.cast(y_true_points, dtype=tf.float32)
        y_pred_points = tf.cast(y_pred_points, dtype=tf.float32)

        # Handle cases where there are no points in either y_true or y_pred
        if tf.shape(y_true_points)[0] == 0 or tf.shape(y_pred_points)[0] == 0:
            return float('inf')  # Return infinity if there are no points to compare

        # Calculate pairwise distances between points in y_true and y_pred
        dists = tf.norm(tf.expand_dims(y_true_points, 1) - tf.expand_dims(y_pred_points, 0), axis=-1)

        # Hausdorff distance is the maximum of the minimum distances from y_true to y_pred and vice versa
        forward_hd = tf.reduce_max(tf.reduce_min(dists, axis=1))
        backward_hd = tf.reduce_max(tf.reduce_min(dists, axis=0))

        # Return the Hausdorff distance
        return tf.reduce_max([forward_hd, backward_hd]).numpy()


    # Calculate boundary F1 and Hausdorff Distance
    bf_score_tf = boundary_f1_score_tf(y_true_tf, y_pred_tf)
    hd_tf = hausdorff_distance_tf(y_true_tf, y_pred_tf)

    # Store results
    results_tf[f'Sample {i}'] = {
        'Accuracy': accuracy.result().numpy(),
        'Precision': precision.result().numpy(),
        'Recall': recall.result().numpy(),
        'IoU': iou.result().numpy(),
        'BF Score': bf_score_tf,
        'Hausdorff Distance': hd_tf
    }

# Convert results to a DataFrame for better readability
results_tf_df = pd.DataFrame(results_tf).T
results_tf_df.index.name = 'Sample'
results_tf_df.reset_index(inplace=True)
print(results_tf_df)

# Save the DataFrame to an Excel file
# os.makedirs('./results', exist_ok=True)
results_tf_df.to_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Segmentation/binary_TF15_metrics.xlsx', index=False)
# Torch 15 pic

       Sample                    Accuracy                   Precision  \
0    Sample 0 0.9999265670776367187500000 0.4671280384063720703125000   
1    Sample 1 0.9999623298645019531250000 0.4893617033958435058593750   
2    Sample 2 0.9999976158142089843750000 0.0000000000000000000000000   
3    Sample 3 0.9999012947082519531250000 0.0000000000000000000000000   
4    Sample 4 0.9997153282165527343750000 0.7488372325897216796875000   
5    Sample 5 1.0000000000000000000000000 0.0000000000000000000000000   
6    Sample 6 0.9999938011169433593750000 0.8387096524238586425781250   
7    Sample 7 0.9999847412109375000000000 0.7333333492279052734375000   
8    Sample 8 0.9999275207519531250000000 0.1000000014901161193847656   
9    Sample 9 0.9999814033508300781250000 0.5609756112098693847656250   
10  Sample 10 0.9999661445617675781250000 0.5851063728332519531250000   
11  Sample 11 0.9999737739562988281250000 0.3846153914928436279296875   
12  Sample 12 0.9999790191650390625000000 0.6981132

# TF 0-29

In [ ]:
# TF 30

import os
import numpy as np
import pandas as pd
import nibabel as nib
import torch
import tensorflow as tf
from torch.utils.data import Dataset
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy, MeanIoU

np.set_printoptions(precision=25)
pd.options.display.float_format = '{:.25f}'.format

# SegmentationDataset Class (PyTorch)
class SegmentationDataset(Dataset):
    def __init__(self, true_seg_dir, pred_seg_dir, transform=None):
        self.true_seg_dir = true_seg_dir
        self.pred_seg_dir = pred_seg_dir
        self.transform = transform

        # Get all files in directories
        self.true_seg_files = sorted(os.listdir(true_seg_dir))
        self.pred_seg_files = sorted(os.listdir(pred_seg_dir))

        # Extract the common parts (e.g. #LIDC-IDRI#LIDC-IDRI-0001#VISIT 1#.nii.gz)
        self.true_seg_base_names = [self.extract_base_name(file) for file in self.true_seg_files]
        self.pred_seg_base_names = [self.extract_base_name(file) for file in self.pred_seg_files]

        # Ensure that there are matching filenames between true and predicted
        assert set(self.true_seg_base_names) == set(self.pred_seg_base_names), \
            "True and predicted segmentation filenames do not match."

    def __len__(self):
        return len(self.true_seg_files)

    def __getitem__(self, idx):
        base_name = self.true_seg_base_names[idx]
        true_seg_filename = [file for file in self.true_seg_files if self.extract_base_name(file) == base_name][0]
        pred_seg_filename = [file for file in self.pred_seg_files if self.extract_base_name(file) == base_name][0]

        true_seg_path = os.path.join(self.true_seg_dir, true_seg_filename)
        pred_seg_path = os.path.join(self.pred_seg_dir, pred_seg_filename)

        true_seg_image = nib.load(true_seg_path).get_fdata()
        pred_seg_image = nib.load(pred_seg_path).get_fdata()

        # Ensure segmentation masks are binary (0 or 1)
        true_seg_image = np.expand_dims(np.clip(true_seg_image, 0, 1), axis=0)
        pred_seg_image = np.expand_dims(np.clip(pred_seg_image, 0, 1), axis=0)

        sample = {'true_seg': true_seg_image, 'pred_seg': pred_seg_image}

        if self.transform:
            sample = self.transform(sample)

        return sample

    def extract_base_name(self, file_name):
        base_name = file_name.split('#', 1)[1]
        return base_name

class ToTensorAndResize(object):
    def __call__(self, sample):
        true_seg = torch.from_numpy(sample['true_seg']).float()
        pred_seg = torch.from_numpy(sample['pred_seg']).float()

        # Reshape the tensors to target size of 128x128x128 (D, H, W)
        true_seg_resized = self.resize_3d(true_seg, (128, 128, 128))
        pred_seg_resized = self.resize_3d(pred_seg, (128, 128, 128))

        return {'true_seg': true_seg_resized, 'pred_seg': pred_seg_resized}

    def resize_3d(self, volume, target_shape):
        """
        Resize a 3D PyTorch tensor to a target shape using trilinear interpolation.
        :param volume: Input 3D PyTorch tensor
        :param target_shape: Desired shape (D, H, W)
        :return: Resized 3D PyTorch tensor
        """
        # Ensure the tensor has at least 3 spatial dimensions
        if len(volume.shape) == 3:  # Shape [D, H, W]
            volume = volume.unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions [1, 1, D, H, W]
        elif len(volume.shape) == 4:  # Shape [C, D, H, W]
            volume = volume.unsqueeze(0)  # Add batch dimension

        # Check the shape before interpolation to make sure it's valid
        if volume.dim() != 5:
            raise ValueError(f"Expected a 5D tensor, but got {volume.dim()}D tensor with shape {volume.shape}")

        # Perform the interpolation (resizing)
        resized_volume = torch.nn.functional.interpolate(volume, size=target_shape, mode='trilinear', align_corners=False)

        # Remove the batch and channel dimensions
        return resized_volume.squeeze(0).squeeze(0)  # Back to shape [D, H, W]


# Load data in PyTorch
true_segmentations_dir = '/content/drive/MyDrive/Mr-Alizadeh/Train-760/CT-segmentation'
predicted_masks_dir = '/content/drive/MyDrive/Mr-Alizadeh/Reconnet_predicted_masks'

# Create dataset (PyTorch)
dataset = SegmentationDataset(true_seg_dir=true_segmentations_dir, pred_seg_dir=predicted_masks_dir, transform=ToTensorAndResize())

# Initialize the results dictionary for TensorFlow metrics
results_tf = {}

# Iterate through the dataset and calculate metrics using TensorFlow
for i in range(30):
    sample = dataset[i]

    # Convert PyTorch tensors to NumPy and then TensorFlow tensors
    y_true_np = sample['true_seg'].numpy()
    y_pred_np = sample['pred_seg'].numpy()

    y_true_tf = tf.convert_to_tensor(y_true_np, dtype=tf.float32)
    y_pred_tf = tf.convert_to_tensor(y_pred_np, dtype=tf.float32)

    # Flatten the tensors for certain metrics
    y_true_flat = tf.reshape(y_true_tf, [-1])
    y_pred_flat = tf.reshape(y_pred_tf, [-1])

    # Convert to binary format for metrics
    y_true_bin = tf.cast(y_true_flat, dtype=tf.int32)
    y_pred_bin = tf.cast(y_pred_flat, dtype=tf.int32)

    # TensorFlow/Keras metrics
    accuracy = BinaryAccuracy()
    precision = Precision()
    recall = Recall()
    iou = MeanIoU(num_classes=2)

    # Update and compute the metrics
    accuracy.update_state(y_true_bin, y_pred_bin)
    precision.update_state(y_true_bin, y_pred_bin)
    recall.update_state(y_true_bin, y_pred_bin)
    iou.update_state(y_true_bin, y_pred_bin)

    # Calculate boundary F1 and Hausdorff Distance
    def extract_boundaries_tf(mask):
        eroded_mask = tf.nn.max_pool3d(mask[None, None, ...], ksize=3, strides=1, padding='SAME')[0, 0]
        boundary = tf.cast(tf.not_equal(mask, eroded_mask), dtype=tf.float32)
        return boundary

    def boundary_f1_score_tf(y_true, y_pred, dilation_radius=1):
        y_true_boundary = extract_boundaries_tf(y_true)
        y_pred_boundary = extract_boundaries_tf(y_pred)
        y_true_boundary_flat = tf.reshape(y_true_boundary, [-1])
        y_pred_boundary_flat = tf.reshape(y_pred_boundary, [-1])
        precision_value = tf.reduce_sum(y_true_boundary_flat * y_pred_boundary_flat) / (tf.reduce_sum(y_pred_boundary_flat) + 1e-6)
        recall_value = tf.reduce_sum(y_true_boundary_flat * y_pred_boundary_flat) / (tf.reduce_sum(y_true_boundary_flat) + 1e-6)
        bf_score = 2 * (precision_value * recall_value) / (precision_value + recall_value + 1e-6)
        return bf_score.numpy()

    def hausdorff_distance_tf(y_true, y_pred):
        """
        Calculate the Hausdorff Distance between two 3D binary masks.
        :param y_true: Ground truth mask
        :param y_pred: Predicted mask
        :return: Hausdorff Distance
        """
        # Get the indices of the true and predicted points
        y_true_points = tf.where(tf.equal(y_true, 1))
        y_pred_points = tf.where(tf.equal(y_pred, 1))

        # Cast the points to float32 for distance calculation
        y_true_points = tf.cast(y_true_points, dtype=tf.float32)
        y_pred_points = tf.cast(y_pred_points, dtype=tf.float32)

        # Handle cases where there are no points in either y_true or y_pred
        if tf.shape(y_true_points)[0] == 0 or tf.shape(y_pred_points)[0] == 0:
            return float('inf')  # Return infinity if there are no points to compare

        # Calculate pairwise distances between points in y_true and y_pred
        dists = tf.norm(tf.expand_dims(y_true_points, 1) - tf.expand_dims(y_pred_points, 0), axis=-1)

        # Hausdorff distance is the maximum of the minimum distances from y_true to y_pred and vice versa
        forward_hd = tf.reduce_max(tf.reduce_min(dists, axis=1))
        backward_hd = tf.reduce_max(tf.reduce_min(dists, axis=0))

        # Return the Hausdorff distance
        return tf.reduce_max([forward_hd, backward_hd]).numpy()


    # Calculate boundary F1 and Hausdorff Distance
    bf_score_tf = boundary_f1_score_tf(y_true_tf, y_pred_tf)
    hd_tf = hausdorff_distance_tf(y_true_tf, y_pred_tf)

    # Store results
    results_tf[f'Sample {i}'] = {
        'Accuracy': accuracy.result().numpy(),
        'Precision': precision.result().numpy(),
        'Recall': recall.result().numpy(),
        'IoU': iou.result().numpy(),
        'BF Score': bf_score_tf,
        'Hausdorff Distance': hd_tf
    }

# Convert results to a DataFrame for better readability
results_tf_df = pd.DataFrame(results_tf).T
results_tf_df.index.name = 'Sample'
results_tf_df.reset_index(inplace=True)
print(results_tf_df)

# Save the DataFrame to an Excel file
# os.makedirs('./results', exist_ok=True)
results_tf_df.to_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Segmentation/binary_TF30.xlsx', index=False)
# Torch 30

       Sample                    Accuracy                   Precision  \
0    Sample 0 0.9999265670776367187500000 0.4671280384063720703125000   
1    Sample 1 0.9999623298645019531250000 0.4893617033958435058593750   
2    Sample 2 0.9999976158142089843750000 0.0000000000000000000000000   
3    Sample 3 0.9999012947082519531250000 0.0000000000000000000000000   
4    Sample 4 0.9997153282165527343750000 0.7488372325897216796875000   
5    Sample 5 1.0000000000000000000000000 0.0000000000000000000000000   
6    Sample 6 0.9999938011169433593750000 0.8387096524238586425781250   
7    Sample 7 0.9999847412109375000000000 0.7333333492279052734375000   
8    Sample 8 0.9999275207519531250000000 0.1000000014901161193847656   
9    Sample 9 0.9999814033508300781250000 0.5609756112098693847656250   
10  Sample 10 0.9999661445617675781250000 0.5851063728332519531250000   
11  Sample 11 0.9999737739562988281250000 0.3846153914928436279296875   
12  Sample 12 0.9999790191650390625000000 0.6981132

# TF 30-59

In [ ]:
# TF 30-59

import os
import numpy as np
import pandas as pd
import nibabel as nib
import torch
import tensorflow as tf
from torch.utils.data import Dataset
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy, MeanIoU

np.set_printoptions(precision=25)
pd.options.display.float_format = '{:.25f}'.format

# SegmentationDataset Class (PyTorch)
class SegmentationDataset(Dataset):
    def __init__(self, true_seg_dir, pred_seg_dir, transform=None):
        self.true_seg_dir = true_seg_dir
        self.pred_seg_dir = pred_seg_dir
        self.transform = transform

        # Get all files in directories
        self.true_seg_files = sorted(os.listdir(true_seg_dir))
        self.pred_seg_files = sorted(os.listdir(pred_seg_dir))

        # Extract the common parts (e.g. #LIDC-IDRI#LIDC-IDRI-0001#VISIT 1#.nii.gz)
        self.true_seg_base_names = [self.extract_base_name(file) for file in self.true_seg_files]
        self.pred_seg_base_names = [self.extract_base_name(file) for file in self.pred_seg_files]

        # Ensure that there are matching filenames between true and predicted
        assert set(self.true_seg_base_names) == set(self.pred_seg_base_names), \
            "True and predicted segmentation filenames do not match."

    def __len__(self):
        return len(self.true_seg_files)

    def __getitem__(self, idx):
        base_name = self.true_seg_base_names[idx]
        true_seg_filename = [file for file in self.true_seg_files if self.extract_base_name(file) == base_name][0]
        pred_seg_filename = [file for file in self.pred_seg_files if self.extract_base_name(file) == base_name][0]

        true_seg_path = os.path.join(self.true_seg_dir, true_seg_filename)
        pred_seg_path = os.path.join(self.pred_seg_dir, pred_seg_filename)

        true_seg_image = nib.load(true_seg_path).get_fdata()
        pred_seg_image = nib.load(pred_seg_path).get_fdata()

        # Ensure segmentation masks are binary (0 or 1)
        true_seg_image = np.expand_dims(np.clip(true_seg_image, 0, 1), axis=0)
        pred_seg_image = np.expand_dims(np.clip(pred_seg_image, 0, 1), axis=0)

        sample = {'true_seg': true_seg_image, 'pred_seg': pred_seg_image}

        if self.transform:
            sample = self.transform(sample)

        return sample

    def extract_base_name(self, file_name):
        base_name = file_name.split('#', 1)[1]
        return base_name

class ToTensorAndResize(object):
    def __call__(self, sample):
        true_seg = torch.from_numpy(sample['true_seg']).float()
        pred_seg = torch.from_numpy(sample['pred_seg']).float()

        # Reshape the tensors to target size of 128x128x128 (D, H, W)
        true_seg_resized = self.resize_3d(true_seg, (128, 128, 128))
        pred_seg_resized = self.resize_3d(pred_seg, (128, 128, 128))

        return {'true_seg': true_seg_resized, 'pred_seg': pred_seg_resized}

    def resize_3d(self, volume, target_shape):
        """
        Resize a 3D PyTorch tensor to a target shape using trilinear interpolation.
        :param volume: Input 3D PyTorch tensor
        :param target_shape: Desired shape (D, H, W)
        :return: Resized 3D PyTorch tensor
        """
        # Ensure the tensor has at least 3 spatial dimensions
        if len(volume.shape) == 3:  # Shape [D, H, W]
            volume = volume.unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions [1, 1, D, H, W]
        elif len(volume.shape) == 4:  # Shape [C, D, H, W]
            volume = volume.unsqueeze(0)  # Add batch dimension

        # Check the shape before interpolation to make sure it's valid
        if volume.dim() != 5:
            raise ValueError(f"Expected a 5D tensor, but got {volume.dim()}D tensor with shape {volume.shape}")

        # Perform the interpolation (resizing)
        resized_volume = torch.nn.functional.interpolate(volume, size=target_shape, mode='trilinear', align_corners=False)

        # Remove the batch and channel dimensions
        return resized_volume.squeeze(0).squeeze(0)  # Back to shape [D, H, W]


# Load data in PyTorch
true_segmentations_dir = '/content/drive/MyDrive/Mr-Alizadeh/Train-760/CT-segmentation'
predicted_masks_dir = '/content/drive/MyDrive/Mr-Alizadeh/Reconnet_predicted_masks'

# Create dataset (PyTorch)
dataset = SegmentationDataset(true_seg_dir=true_segmentations_dir, pred_seg_dir=predicted_masks_dir, transform=ToTensorAndResize())

# Initialize the results dictionary for TensorFlow metrics
results_tf = {}

# Iterate through the dataset and calculate metrics using TensorFlow
for i in range(30,60):
    sample = dataset[i]

    # Convert PyTorch tensors to NumPy and then TensorFlow tensors
    y_true_np = sample['true_seg'].numpy()
    y_pred_np = sample['pred_seg'].numpy()

    y_true_tf = tf.convert_to_tensor(y_true_np, dtype=tf.float32)
    y_pred_tf = tf.convert_to_tensor(y_pred_np, dtype=tf.float32)

    # Flatten the tensors for certain metrics
    y_true_flat = tf.reshape(y_true_tf, [-1])
    y_pred_flat = tf.reshape(y_pred_tf, [-1])

    # Convert to binary format for metrics
    y_true_bin = tf.cast(y_true_flat, dtype=tf.int32)
    y_pred_bin = tf.cast(y_pred_flat, dtype=tf.int32)

    # TensorFlow/Keras metrics
    accuracy = BinaryAccuracy()
    precision = Precision()
    recall = Recall()
    iou = MeanIoU(num_classes=2)

    # Update and compute the metrics
    accuracy.update_state(y_true_bin, y_pred_bin)
    precision.update_state(y_true_bin, y_pred_bin)
    recall.update_state(y_true_bin, y_pred_bin)
    iou.update_state(y_true_bin, y_pred_bin)

    # Calculate boundary F1 and Hausdorff Distance
    def extract_boundaries_tf(mask):
        eroded_mask = tf.nn.max_pool3d(mask[None, None, ...], ksize=3, strides=1, padding='SAME')[0, 0]
        boundary = tf.cast(tf.not_equal(mask, eroded_mask), dtype=tf.float32)
        return boundary

    def boundary_f1_score_tf(y_true, y_pred, dilation_radius=1):
        y_true_boundary = extract_boundaries_tf(y_true)
        y_pred_boundary = extract_boundaries_tf(y_pred)
        y_true_boundary_flat = tf.reshape(y_true_boundary, [-1])
        y_pred_boundary_flat = tf.reshape(y_pred_boundary, [-1])
        precision_value = tf.reduce_sum(y_true_boundary_flat * y_pred_boundary_flat) / (tf.reduce_sum(y_pred_boundary_flat) + 1e-6)
        recall_value = tf.reduce_sum(y_true_boundary_flat * y_pred_boundary_flat) / (tf.reduce_sum(y_true_boundary_flat) + 1e-6)
        bf_score = 2 * (precision_value * recall_value) / (precision_value + recall_value + 1e-6)
        return bf_score.numpy()

    def hausdorff_distance_tf(y_true, y_pred):
        """
        Calculate the Hausdorff Distance between two 3D binary masks.
        :param y_true: Ground truth mask
        :param y_pred: Predicted mask
        :return: Hausdorff Distance
        """
        # Get the indices of the true and predicted points
        y_true_points = tf.where(tf.equal(y_true, 1))
        y_pred_points = tf.where(tf.equal(y_pred, 1))

        # Cast the points to float32 for distance calculation
        y_true_points = tf.cast(y_true_points, dtype=tf.float32)
        y_pred_points = tf.cast(y_pred_points, dtype=tf.float32)

        # Handle cases where there are no points in either y_true or y_pred
        if tf.shape(y_true_points)[0] == 0 or tf.shape(y_pred_points)[0] == 0:
            return float('inf')  # Return infinity if there are no points to compare

        # Calculate pairwise distances between points in y_true and y_pred
        dists = tf.norm(tf.expand_dims(y_true_points, 1) - tf.expand_dims(y_pred_points, 0), axis=-1)

        # Hausdorff distance is the maximum of the minimum distances from y_true to y_pred and vice versa
        forward_hd = tf.reduce_max(tf.reduce_min(dists, axis=1))
        backward_hd = tf.reduce_max(tf.reduce_min(dists, axis=0))

        # Return the Hausdorff distance
        return tf.reduce_max([forward_hd, backward_hd]).numpy()


    # Calculate boundary F1 and Hausdorff Distance
    bf_score_tf = boundary_f1_score_tf(y_true_tf, y_pred_tf)
    hd_tf = hausdorff_distance_tf(y_true_tf, y_pred_tf)

    # Store results
    results_tf[f'Sample {i}'] = {
        'Accuracy': accuracy.result().numpy(),
        'Precision': precision.result().numpy(),
        'Recall': recall.result().numpy(),
        'IoU': iou.result().numpy(),
        'BF Score': bf_score_tf,
        'Hausdorff Distance': hd_tf
    }

# Convert results to a DataFrame for better readability
results_tf_df = pd.DataFrame(results_tf).T
results_tf_df.index.name = 'Sample'
results_tf_df.reset_index(inplace=True)
print(results_tf_df)

# Save the DataFrame to an Excel file
# os.makedirs('./results', exist_ok=True)
results_tf_df.to_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Segmentation/binary_TF30-59.xlsx', index=False)
# Torch 30-59

       Sample                    Accuracy                   Precision  \
0   Sample 30 0.9999761581420898437500000 0.4588235318660736083984375   
1   Sample 31 0.9999928474426269531250000 0.3750000000000000000000000   
2   Sample 32 0.9999704360961914062500000 0.7372881174087524414062500   
3   Sample 33 0.9999876022338867187500000 0.4222222268581390380859375   
4   Sample 34 0.9999856948852539062500000 0.4444444477558135986328125   
5   Sample 35 0.9999446868896484375000000 0.6452599167823791503906250   
6   Sample 36 0.9999842643737792968750000 0.5925925970077514648437500   
7   Sample 37 0.9999704360961914062500000 0.3015873134136199951171875   
8   Sample 38 0.9999938011169433593750000 0.0000000000000000000000000   
9   Sample 39 0.9999709129333496093750000 0.6114649772644042968750000   
10  Sample 40 0.9998803138732910156250000 0.5030060410499572753906250   
11  Sample 41 0.9997115135192871093750000 0.8232174515724182128906250   
12  Sample 42 0.9998850822448730468750000 0.3852040

# TF 60-99

In [ ]:
# TF 60-99

import os
import numpy as np
import pandas as pd
import nibabel as nib
import torch
import tensorflow as tf
from torch.utils.data import Dataset
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy, MeanIoU

np.set_printoptions(precision=25)
pd.options.display.float_format = '{:.25f}'.format

# SegmentationDataset Class (PyTorch)
class SegmentationDataset(Dataset):
    def __init__(self, true_seg_dir, pred_seg_dir, transform=None):
        self.true_seg_dir = true_seg_dir
        self.pred_seg_dir = pred_seg_dir
        self.transform = transform

        # Get all files in directories
        self.true_seg_files = sorted(os.listdir(true_seg_dir))
        self.pred_seg_files = sorted(os.listdir(pred_seg_dir))

        # Extract the common parts (e.g. #LIDC-IDRI#LIDC-IDRI-0001#VISIT 1#.nii.gz)
        self.true_seg_base_names = [self.extract_base_name(file) for file in self.true_seg_files]
        self.pred_seg_base_names = [self.extract_base_name(file) for file in self.pred_seg_files]

        # Ensure that there are matching filenames between true and predicted
        assert set(self.true_seg_base_names) == set(self.pred_seg_base_names), \
            "True and predicted segmentation filenames do not match."

    def __len__(self):
        return len(self.true_seg_files)

    def __getitem__(self, idx):
        base_name = self.true_seg_base_names[idx]
        true_seg_filename = [file for file in self.true_seg_files if self.extract_base_name(file) == base_name][0]
        pred_seg_filename = [file for file in self.pred_seg_files if self.extract_base_name(file) == base_name][0]

        true_seg_path = os.path.join(self.true_seg_dir, true_seg_filename)
        pred_seg_path = os.path.join(self.pred_seg_dir, pred_seg_filename)

        true_seg_image = nib.load(true_seg_path).get_fdata()
        pred_seg_image = nib.load(pred_seg_path).get_fdata()

        # Ensure segmentation masks are binary (0 or 1)
        true_seg_image = np.expand_dims(np.clip(true_seg_image, 0, 1), axis=0)
        pred_seg_image = np.expand_dims(np.clip(pred_seg_image, 0, 1), axis=0)

        sample = {'true_seg': true_seg_image, 'pred_seg': pred_seg_image}

        if self.transform:
            sample = self.transform(sample)

        return sample

    def extract_base_name(self, file_name):
        base_name = file_name.split('#', 1)[1]
        return base_name

class ToTensorAndResize(object):
    def __call__(self, sample):
        true_seg = torch.from_numpy(sample['true_seg']).float()
        pred_seg = torch.from_numpy(sample['pred_seg']).float()

        # Reshape the tensors to target size of 128x128x128 (D, H, W)
        true_seg_resized = self.resize_3d(true_seg, (128, 128, 128))
        pred_seg_resized = self.resize_3d(pred_seg, (128, 128, 128))

        return {'true_seg': true_seg_resized, 'pred_seg': pred_seg_resized}

    def resize_3d(self, volume, target_shape):
        """
        Resize a 3D PyTorch tensor to a target shape using trilinear interpolation.
        :param volume: Input 3D PyTorch tensor
        :param target_shape: Desired shape (D, H, W)
        :return: Resized 3D PyTorch tensor
        """
        # Ensure the tensor has at least 3 spatial dimensions
        if len(volume.shape) == 3:  # Shape [D, H, W]
            volume = volume.unsqueeze(0).unsqueeze(0)  # Add batch and channel dimensions [1, 1, D, H, W]
        elif len(volume.shape) == 4:  # Shape [C, D, H, W]
            volume = volume.unsqueeze(0)  # Add batch dimension

        # Check the shape before interpolation to make sure it's valid
        if volume.dim() != 5:
            raise ValueError(f"Expected a 5D tensor, but got {volume.dim()}D tensor with shape {volume.shape}")

        # Perform the interpolation (resizing)
        resized_volume = torch.nn.functional.interpolate(volume, size=target_shape, mode='trilinear', align_corners=False)

        # Remove the batch and channel dimensions
        return resized_volume.squeeze(0).squeeze(0)  # Back to shape [D, H, W]


# Load data in PyTorch
true_segmentations_dir = '/content/drive/MyDrive/Mr-Alizadeh/Train-760/CT-segmentation'
predicted_masks_dir = '/content/drive/MyDrive/Mr-Alizadeh/Reconnet_predicted_masks'

# Create dataset (PyTorch)
dataset = SegmentationDataset(true_seg_dir=true_segmentations_dir, pred_seg_dir=predicted_masks_dir, transform=ToTensorAndResize())

# Initialize the results dictionary for TensorFlow metrics
results_tf = {}

# Iterate through the dataset and calculate metrics using TensorFlow
for i in range(60,100):
    sample = dataset[i]

    # Convert PyTorch tensors to NumPy and then TensorFlow tensors
    y_true_np = sample['true_seg'].numpy()
    y_pred_np = sample['pred_seg'].numpy()

    y_true_tf = tf.convert_to_tensor(y_true_np, dtype=tf.float32)
    y_pred_tf = tf.convert_to_tensor(y_pred_np, dtype=tf.float32)

    # Flatten the tensors for certain metrics
    y_true_flat = tf.reshape(y_true_tf, [-1])
    y_pred_flat = tf.reshape(y_pred_tf, [-1])

    # Convert to binary format for metrics
    y_true_bin = tf.cast(y_true_flat, dtype=tf.int32)
    y_pred_bin = tf.cast(y_pred_flat, dtype=tf.int32)

    # TensorFlow/Keras metrics
    accuracy = BinaryAccuracy()
    precision = Precision()
    recall = Recall()
    iou = MeanIoU(num_classes=2)

    # Update and compute the metrics
    accuracy.update_state(y_true_bin, y_pred_bin)
    precision.update_state(y_true_bin, y_pred_bin)
    recall.update_state(y_true_bin, y_pred_bin)
    iou.update_state(y_true_bin, y_pred_bin)

    # Calculate boundary F1 and Hausdorff Distance
    def extract_boundaries_tf(mask):
        eroded_mask = tf.nn.max_pool3d(mask[None, None, ...], ksize=3, strides=1, padding='SAME')[0, 0]
        boundary = tf.cast(tf.not_equal(mask, eroded_mask), dtype=tf.float32)
        return boundary

    def boundary_f1_score_tf(y_true, y_pred, dilation_radius=1):
        y_true_boundary = extract_boundaries_tf(y_true)
        y_pred_boundary = extract_boundaries_tf(y_pred)
        y_true_boundary_flat = tf.reshape(y_true_boundary, [-1])
        y_pred_boundary_flat = tf.reshape(y_pred_boundary, [-1])
        precision_value = tf.reduce_sum(y_true_boundary_flat * y_pred_boundary_flat) / (tf.reduce_sum(y_pred_boundary_flat) + 1e-6)
        recall_value = tf.reduce_sum(y_true_boundary_flat * y_pred_boundary_flat) / (tf.reduce_sum(y_true_boundary_flat) + 1e-6)
        bf_score = 2 * (precision_value * recall_value) / (precision_value + recall_value + 1e-6)
        return bf_score.numpy()

    def hausdorff_distance_tf(y_true, y_pred):
        """
        Calculate the Hausdorff Distance between two 3D binary masks.
        :param y_true: Ground truth mask
        :param y_pred: Predicted mask
        :return: Hausdorff Distance
        """
        # Get the indices of the true and predicted points
        y_true_points = tf.where(tf.equal(y_true, 1))
        y_pred_points = tf.where(tf.equal(y_pred, 1))

        # Cast the points to float32 for distance calculation
        y_true_points = tf.cast(y_true_points, dtype=tf.float32)
        y_pred_points = tf.cast(y_pred_points, dtype=tf.float32)

        # Handle cases where there are no points in either y_true or y_pred
        if tf.shape(y_true_points)[0] == 0 or tf.shape(y_pred_points)[0] == 0:
            return float('inf')  # Return infinity if there are no points to compare

        # Calculate pairwise distances between points in y_true and y_pred
        dists = tf.norm(tf.expand_dims(y_true_points, 1) - tf.expand_dims(y_pred_points, 0), axis=-1)

        # Hausdorff distance is the maximum of the minimum distances from y_true to y_pred and vice versa
        forward_hd = tf.reduce_max(tf.reduce_min(dists, axis=1))
        backward_hd = tf.reduce_max(tf.reduce_min(dists, axis=0))

        # Return the Hausdorff distance
        return tf.reduce_max([forward_hd, backward_hd]).numpy()


    # Calculate boundary F1 and Hausdorff Distance
    bf_score_tf = boundary_f1_score_tf(y_true_tf, y_pred_tf)
    hd_tf = hausdorff_distance_tf(y_true_tf, y_pred_tf)

    # Store results
    results_tf[f'Sample {i}'] = {
        'Accuracy': accuracy.result().numpy(),
        'Precision': precision.result().numpy(),
        'Recall': recall.result().numpy(),
        'IoU': iou.result().numpy(),
        'BF Score': bf_score_tf,
        'Hausdorff Distance': hd_tf
    }

# Convert results to a DataFrame for better readability
results_tf_df = pd.DataFrame(results_tf).T
results_tf_df.index.name = 'Sample'
results_tf_df.reset_index(inplace=True)
print(results_tf_df)

# Save the DataFrame to an Excel file
# os.makedirs('./results', exist_ok=True)
results_tf_df.to_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Segmentation/binary_TF60-99.xlsx', index=False)
# Torch 60-99

       Sample                    Accuracy                   Precision  \
0   Sample 60 0.9999804496765136718750000 0.4938271641731262207031250   
1   Sample 61 0.9999856948852539062500000 0.7767857313156127929687500   
2   Sample 62 0.9999961853027343750000000 0.1428571492433547973632812   
3   Sample 63 0.9999995231628417968750000 0.0000000000000000000000000   
4   Sample 64 0.9999208450317382812500000 0.9252873659133911132812500   
5   Sample 65 0.9999518394470214843750000 0.4093567132949829101562500   
6   Sample 66 0.9999775886535644531250000 0.7684729099273681640625000   
7   Sample 67 0.9999985694885253906250000 0.4000000059604644775390625   
8   Sample 68 0.9999923706054687500000000 0.2500000000000000000000000   
9   Sample 69 0.9999985694885253906250000 0.0000000000000000000000000   
10  Sample 70 0.9999847412109375000000000 0.8727272748947143554687500   
11  Sample 71 0.9999899864196777343750000 0.0869565233588218688964844   
12  Sample 72 1.0000000000000000000000000 0.0000000

# Torch 760 pic - bach 10

In [ ]:
# Torch 760 pic - bach 10

import pandas as pd
import numpy as np
import torch
import torchmetrics
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import nibabel as nib
import os
from decimal import getcontext, Decimal
from torchmetrics.functional import dice

np.set_printoptions(precision=25)
getcontext().prec = 25
pd.options.display.float_format = '{:.25f}'.format

# Set precision for numpy
np.set_printoptions(precision=25)

# Set precision for pandas
pd.set_option('display.float_format', '{:.25f}'.format)

# SegmentationDataset Class
class SegmentationDataset(Dataset):
    def __init__(self, true_seg_dir, pred_seg_dir, transform=None):
        self.true_seg_dir = true_seg_dir
        self.pred_seg_dir = pred_seg_dir
        self.transform = transform

        # Get all files in directories
        self.true_seg_files = sorted(os.listdir(true_seg_dir))
        self.pred_seg_files = sorted(os.listdir(pred_seg_dir))

        # Extract the common parts (e.g. #LIDC-IDRI#LIDC-IDRI-0001#VISIT 1#.nii.gz)
        self.true_seg_base_names = [self.extract_base_name(file) for file in self.true_seg_files]
        self.pred_seg_base_names = [self.extract_base_name(file) for file in self.pred_seg_files]

        # Ensure that there are matching filenames between true and predicted
        assert set(self.true_seg_base_names) == set(self.pred_seg_base_names), \
            "True and predicted segmentation filenames do not match."

    def __len__(self):
        return len(self.true_seg_files)

    def __getitem__(self, idx):
        # Find the matching true and predicted segmentation files
        base_name = self.true_seg_base_names[idx]
        true_seg_filename = [file for file in self.true_seg_files if self.extract_base_name(file) == base_name][0]
        pred_seg_filename = [file for file in self.pred_seg_files if self.extract_base_name(file) == base_name][0]

        true_seg_path = os.path.join(self.true_seg_dir, true_seg_filename)
        pred_seg_path = os.path.join(self.pred_seg_dir, pred_seg_filename)

        true_seg_image = nib.load(true_seg_path).get_fdata()
        pred_seg_image = nib.load(pred_seg_path).get_fdata()

        # Ensure segmentation masks are binary (if needed)
        true_seg_image = np.expand_dims(np.clip(true_seg_image, 0, 1), axis=0)
        pred_seg_image = np.expand_dims(np.clip(pred_seg_image, 0, 1), axis=0)

        sample = {'true_seg': true_seg_image, 'pred_seg': pred_seg_image}

        if self.transform:
            sample = self.transform(sample)

        return sample

    def extract_base_name(self, file_name):
        base_name = file_name.split('#', 1)[1]
        return base_name


# ToTensor Class
class ToTensor(object):
    def __call__(self, sample):
        true_seg = torch.from_numpy(sample['true_seg'])
        pred_seg = torch.from_numpy(sample['pred_seg'])
        true_seg, pred_seg = true_seg.type(torch.float32), pred_seg.type(torch.float32)

        x, y, z = 256, 256, 128  # Example target shape (D, H, W)
        true_seg_resized = F.interpolate(true_seg.unsqueeze(0), size=(x, y, z), mode='trilinear', align_corners=False).squeeze(0)
        pred_seg_resized = F.interpolate(pred_seg.unsqueeze(0), size=(x, y, z), mode='trilinear', align_corners=False).squeeze(0)

        return {'true_seg': true_seg_resized, 'pred_seg': pred_seg_resized}


# Example usage
true_segmentations_dir = '/content/drive/MyDrive/Mr-Alizadeh/Train-760/CT-segmentation'
predicted_masks_dir = '/content/drive/MyDrive/Mr-Alizadeh/Reconnet_predicted_masks'

# Create dataset and dataloader
dataset = SegmentationDataset(true_seg_dir=true_segmentations_dir, pred_seg_dir=predicted_masks_dir, transform=ToTensor())

# Load data in batches of 10
dataloader = DataLoader(dataset, batch_size=10, shuffle=False)

# Initialize the results dictionary
results2 = {}

# Iterate through the dataloader and calculate metrics
for i, sample_batch in enumerate(dataloader):
    for j in range(len(sample_batch['true_seg'])):
        y_true = sample_batch['true_seg'][j]
        y_pred = sample_batch['pred_seg'][j]

        # Ensure unique samples are being loaded
        print(f"Processing Batch {i}, Sample {j}: y_true.shape = {y_true.shape}, y_pred.shape = {y_pred.shape}")

        # Clone and detach the tensors to avoid any computational graph tracking
        y_true = y_true.clone().detach().type(torch.float32)
        y_pred = y_pred.clone().detach().type(torch.float32)

        # Flatten the images for certain metrics
        y_true_flat = y_true.flatten()
        y_pred_flat = y_pred.flatten()

        # PyTorch metrics
        y_true_torch = y_true_flat.type(torch.int)
        y_pred_torch = y_pred_flat.type(torch.int)
        # Boundary F1 Score and Hausdorff Distance functions
        def extract_boundaries_torch(mask):
            """Extract boundary pixels from a binary segmentation mask using PyTorch."""
            if len(mask.shape) == 3:
                mask = mask.unsqueeze(0).unsqueeze(0)
            if mask.dim() == 5:
                eroded_mask = F.max_pool3d(mask, kernel_size=3, stride=1, padding=1).squeeze()
            else:
                eroded_mask = F.max_pool2d(mask, kernel_size=3, stride=1, padding=1).squeeze()
            boundary = (mask.squeeze() != eroded_mask).float()
            return boundary


        def custom_dilation_torch(mask, kernel_size=3):
            """Dilate the boundary pixels for 3D data."""
            kernel = torch.ones((1, 1, kernel_size, kernel_size, kernel_size), dtype=torch.float32)
            mask = mask.unsqueeze(0).unsqueeze(0)
            dilated = F.conv3d(mask, kernel, padding=kernel_size // 2).squeeze()
            dilated = (dilated > 0).float()
            return dilated


        def boundary_f1_score_torch(y_true, y_pred, dilation_radius=1):
            y_true_boundary = extract_boundaries_torch(y_true)
            y_pred_boundary = extract_boundaries_torch(y_pred)
            y_true_boundary_dilated = custom_dilation_torch(y_true_boundary, kernel_size=2 * dilation_radius + 1)
            y_pred_boundary_dilated = custom_dilation_torch(y_pred_boundary, kernel_size=2 * dilation_radius + 1)
            y_true_boundary_flat = y_true_boundary_dilated.flatten()
            y_pred_boundary_flat = y_pred_boundary_dilated.flatten()
            precision_metric = torchmetrics.Precision(task="binary")
            recall_metric = torchmetrics.Recall(task="binary")
            precision_metric.update(y_pred_boundary_flat, y_true_boundary_flat)
            recall_metric.update(y_pred_boundary_flat, y_true_boundary_flat)
            precision = precision_metric.compute().item()
            recall = recall_metric.compute().item()
            if precision + recall > 0:
                bf1_score = 2 * (precision * recall) / (precision + recall)
            else:
                bf1_score = 0.0
            return bf1_score


        def hausdorff_distance_torch(y_true, y_pred):
            y_true_boundary = extract_boundaries_torch(y_true)
            y_pred_boundary = extract_boundaries_torch(y_pred)
            y_true_coords = torch.nonzero(y_true_boundary)
            y_pred_coords = torch.nonzero(y_pred_boundary)
            if len(y_true_coords) == 0 or len(y_pred_coords) == 0:
                return torch.tensor(float('inf'))
            diff = torch.cdist(y_true_coords.float(), y_pred_coords.float(), p=2)
            hd = torch.max(torch.min(diff, dim=0)[0].max(), torch.min(diff, dim=1)[0].max())
            return hd.item()
        accuracy_metric = torchmetrics.Accuracy(task="binary")
        precision_metric = torchmetrics.Precision(task="binary")
        recall_metric = torchmetrics.Recall(task="binary")
        f1_metric = torchmetrics.F1Score(task="binary")
        iou_metric = torchmetrics.JaccardIndex(task="binary")

        accuracy_metric.update(y_pred_torch, y_true_torch)
        precision_metric.update(y_pred_torch, y_true_torch)
        recall_metric.update(y_pred_torch, y_true_torch)
        f1_metric.update(y_pred_torch, y_true_torch)
        iou_metric.update(y_pred_torch, y_true_torch)

        pt_accuracy = accuracy_metric.compute().item()
        pt_precision = precision_metric.compute().item()
        pt_recall = recall_metric.compute().item()
        pt_f1 = f1_metric.compute().item()
        pt_iou = iou_metric.compute().item()

        bf_score_torch = boundary_f1_score_torch(y_true, y_pred)
        hd_torch = hausdorff_distance_torch(y_true, y_pred)

        # Save all results for each sample in the batch
        results2[f'Batch {i}, Sample {j}'] = {
            'Accuracy': pt_accuracy,
            'Precision': pt_precision,
            'Recall': pt_recall,
            'F1 Score': pt_f1,
            'IoU': pt_iou,
            'Dice Coefficient': dice(y_pred_torch, y_true_torch).item(),
            'BF Score': bf_score_torch,
            'Hausdorff Distance': hd_torch
        }
        results2_df = pd.DataFrame(results2).T
        results2_df.index.name = 'Sample'
        results2_df.reset_index(inplace=True)
        print(results2_df)
        results2_df.to_excel(f'/content/drive/MyDrive/Loss Function Evaluation/Data/Segmentation/binary_TORCH_Bach_metrics{i}.xlsx', index=False)


# Convert results to a DataFrame for better readability
results2_df = pd.DataFrame(results2).T
results2_df.index.name = 'Sample'
results2_df.reset_index(inplace=True)
print(results2_df)

# Save the DataFrame to an Excel file
results2_df.to_excel('/content/drive/MyDrive/Loss Function Evaluation/Data/Segmentation/binary_TORCH_Bach_metrics.xlsx', index=False)





Streaming output truncated to the last 5000 lines.
685 0.0000000000000000000000000 0.0000000000000000000000000   
686 0.0000000000000000000000000 0.0000000000000000000000000   

                       F1 Score                         IoU  \
0   0.8071262836456298828125000 0.6766234040260314941406250   
1   0.6813559532165527343750000 0.5167095065116882324218750   
2   0.0000000000000000000000000 0.0000000000000000000000000   
3   0.0000000000000000000000000 0.0000000000000000000000000   
4   0.5640742778778076171875000 0.3928297162055969238281250   
..                          ...                         ...   
682 0.0000000000000000000000000 0.0000000000000000000000000   
683 0.1366247534751892089843750 0.0733211115002632141113281   
684 0.1921387016773223876953125 0.1062795594334602355957031   
685 0.0000000000000000000000000 0.0000000000000000000000000   
686 0.0000000000000000000000000 0.0000000000000000000000000   

               Dice Coefficient                    BF Score  \
0 